# Preparing environment

In [1]:
import requests
import json
import pandas as pd
import psycopg2
import numpy as np
import logging
import sys


# Set up logging

In [2]:
class LevelRangeFilter(logging.Filter):
    """
    A logging filter that allows messages within a specified range of logging levels to pass through.

    Attributes:
        min_level (int): The minimum logging level that the filter allows.
        max_level (int): The maximum logging level that the filter allows.
    """
    def __init__(self, min_level, max_level):
        """
        Initializes the filter with minimum and maximum logging levels.

        Args:
            min_level (int): The minimum logging level.
            max_level (int): The maximum logging level.
        """
        super().__init__()
        self.min_level = min_level
        self.max_level = max_level

    def filter(self, record):
        """
        Determines if the specified record should be logged based on its level.

        Args:
            record (LogRecord): The log record to be checked.

        Returns:
            bool: True if the record's level is within the specified range, False otherwise.
        """
        # Filter records that are not in the specified level range
        return self.min_level <= record.levelno <= self.max_level


# Create a logger for this module
main_logger = logging.getLogger(__name__)

# Set the logging level for the logger
main_logger.setLevel(logging.DEBUG)
# Setting the logger level to DEBUG means that all log messages, regardless of their severity, will be processed and output by this logger.

# Create handlers
success_handler = logging.FileHandler('etl_success.log')
debug_handler = logging.FileHandler('etl_debug.log')
error_handler = logging.FileHandler('etl_errors.log')
console_handler = logging.StreamHandler()  # This handler will send logs to the console

# Create formatters and add them to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
success_handler.setFormatter(formatter)
debug_handler.setFormatter(formatter)
error_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Create filters to the handlers
success_filter = LevelRangeFilter(logging.INFO, logging.WARNING)
error_filter = LevelRangeFilter(logging.WARNING, logging.CRITICAL)
debug_filter = LevelRangeFilter(logging.DEBUG, logging.CRITICAL)
console_filter = LevelRangeFilter(logging.INFO, logging.CRITICAL)

# Add filters to the handlers
success_handler.addFilter(success_filter)
error_handler.addFilter(error_filter)
debug_handler.addFilter(debug_filter)
console_handler.addFilter(console_filter)

# Clear any existing handlers attached to the logger
main_logger.handlers.clear()

# check if the logger currently has any handlers attached to it.
# logger.handlers will return a list of handlers attached to logger
if not main_logger.handlers:  #If this list is empty  it means no handlers are currently attached to the logger.
    # Add handlers to the logger
    main_logger.addHandler(success_handler)
    main_logger.addHandler(debug_handler)
    main_logger.addHandler(error_handler)
    main_logger.addHandler(console_handler)

# Disable propagation to avoid double logging
main_logger.propagate = False


# Extract

**API DOC**
https://spoonacular.com/application/frontend/downloads/spoonacular-api-slides.pdf

**API Link** https://spoonacular.com/food-api

In [ ]:
# Function to fetch recipes from the Spoonacular API
def fetch_recipes(url, api_key, offset, number):
    params = {
        'apiKey': api_key, 
        'number': number,  # The number of records to fetch in one request.
        'offset': offset,   # The number of records to skip (used for pagination).
        'addRecipeInformation': True,
        'instructionsRequired': True,
        'fillIngredients': True,
        'addRecipeInstructions': True,
        'addRecipeNutrition': True
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error for bad status codes
    data = response.json() 
    return data

main_logger.info('Starting data extraction')
# Set your API key
API_KEY = 'c724b4224ab24504ad57792b2acd2605'

# Set your API URL
API_URL='https://api.spoonacular.com/recipes/complexSearch'

# Check if the offset and titles_set arguments are provided
if len(sys.argv) < 3:
    main_logger.error('offset or titles_set argument is missing.')
    sys.exit(1)

# Get the offset and titles_set from the command-line arguments
try:
    offset = int(sys.argv[1])
    main_logger.info(f"initial offset: {offset}")
except Exception as e:
    main_logger.error(f"Failed to extract initial offset:{e}", exc_info=True)

try:
    titles_set = set(sys.argv[2].split('\n')) if sys.argv[2] else set()
    main_logger.debug(f"initial titles_set: {titles_set}")
except Exception as e:
    main_logger.error(f"Failed to extract initial offset:{e}", exc_info=True)

# Fetch 50 recipes
try:
    NUMBER = 50
    execution_batch = fetch_recipes(API_URL, API_KEY, offset, NUMBER)
    main_logger.debug(f"Extracted {NUMBER} recipes")
except Exception as e:
    main_logger.error(f"Failed to extract recipes:{e}", exc_info=True)

# Filter out recipes that have titles already in titles_set
try:
    filtered_recipes = [recipe for recipe in execution_batch['results'] if recipe['title'] not in titles_set]
    main_logger.debug("Filtered recipes")
except Exception as e:
    main_logger.error(f"Failed to Filter recipes:{e}", exc_info=True)

# Extract titles of the execution batch to ensure the next batch is different
try:
    unique_new_titles = {recipe['title'] for recipe in filtered_recipes}
    titles_set.update(unique_new_titles)
    main_logger.debug("titles_set updated")
except Exception as e:
    main_logger.error(f"Failed to update titles_set:{e}", exc_info=True)

offset += NUMBER
main_logger.info(f"intermediate offset: {offset}")

# Check if we need more recipes to reach 100 unique recipes
while len(filtered_recipes) < NUMBER:
    try:
        remaining_needed = NUMBER - len(filtered_recipes)
        more_recipes = fetch_recipes(api_key, offset, remaining_needed)
        main_logger.debug(f"Extracted more {remaining_needed} recipes")
        filtered_more_recipes = [recipe for recipe in more_recipes['results'] if recipe['title'] not in titles_set]
        main_logger.debug("Filtered more recipes")
        filtered_recipes.extend(filtered_more_recipes)
        main_logger.debug("Extended filtered_recipes")
        unique_new_titles = {recipe['title'] for recipe in filtered_more_recipes}  # Ensure only needed titles are added
        titles_set.update(unique_new_titles)
        main_logger.debug("titles_set updated")
        offset += len(more_recipes) # Because the first extracted recipes are already used
        main_logger.info(f"intermediate offset: {offset}")
        if len(filtered_more_recipes) == 0:
            break  # Stop if there are no more unique recipes available
    except Exception as e:
        main_logger.error(f"Failed to extract more recipes: {e}", exc_info=True)

# Write the updated offset to the new_offset.txt file, each title on a new line
try:
    with open('new_offset.txt', 'w') as file:
        file.write(f"{offset}\n")     
    main_logger.debug('offset data written to file')     
except Exception as e:
    main_logger.error(f"Failed to write offset data to file:{e}", exc_info=True)

# Write the updated titles_set to the new_titles_set.txt file, each title on a new line
try:
    with open('new_titles_set.txt', 'w') as file:
        for title in titles_set:
            file.write(f"{title}\n")
    main_logger.debug('Unique titles written to file') 
except Exception as e:
    main_logger.error(f"Failed to write Unique titles to file:{e}", exc_info=True)

# Write the filtered_recipes to the new_recipes.txt file
try:
    with open('new_recipes.txt', 'w') as file:
        json.dump(filtered_recipes, file, indent=4)  
        #The json.dump function in Python is used to serialize a Python object (in this case, a list of dictionaries) to a JSON formatted string and write it to a file. 
    main_logger.info('Extracted data written to file')   
except Exception as e:
    main_logger.error(f"Failed to write Extracted data to file:{e}", exc_info=True)


# Transform

## Json to Pandas
you can import any json data into pandas, the keys will become columns names

In [33]:
try:
    with open('recipes_to_transform.txt', 'r') as file:
        data = json.load(file)
    main_logger.info('Successfully loaded data from json_file')
    df = pd.DataFrame(data)
    main_logger.debug(df.head())
    main_logger.debug(df.info())
except Exception as e:
    main_logger.error(f"Error loading data from recipes_to_transform.txt: {e}", exc_info=True)

2024-07-20 18:35:38,366 - __main__ - INFO - Successfully loaded data from json_file


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   vegetarian                50 non-null     bool   
 1   vegan                     50 non-null     bool   
 2   glutenFree                50 non-null     bool   
 3   dairyFree                 50 non-null     bool   
 4   veryHealthy               50 non-null     bool   
 5   cheap                     50 non-null     bool   
 6   veryPopular               50 non-null     bool   
 7   sustainable               50 non-null     bool   
 8   lowFodmap                 50 non-null     bool   
 9   weightWatcherSmartPoints  50 non-null     int64  
 10  gaps                      50 non-null     object 
 11  preparationMinutes        6 non-null      float64
 12  cookingMinutes            6 non-null      float64
 13  aggregateLikes            50 non-null     int64  
 14  healthScore 

In [34]:
# Show all columns
pd.set_option('display.max_columns', None)
# Display the first few rows of the DataFrame to verify its structure
df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,gaps,preparationMinutes,cookingMinutes,aggregateLikes,healthScore,creditsText,sourceName,pricePerServing,extendedIngredients,id,title,readyInMinutes,servings,sourceUrl,image,imageType,nutrition,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,usedIngredientCount,missedIngredientCount,missedIngredients,likes,usedIngredients,unusedIngredients,license,author
0,True,False,True,True,True,False,False,False,False,8,no,NaN,NaN,5,100,foodista.com,foodista.com,262.42,"[{'id': 6194, 'aisle': 'Canned and Jarred', 'i...",660405,Smoky Black Bean Soup With Sweet Potato & Kale,630,6,https://www.foodista.com/recipe/B8SQD87X/smoky...,https://img.spoonacular.com/recipes/660405-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",You can never have too many main course recipe...,[],"[lunch, soup, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",96.104034,https://spoonacular.com/smoky-black-bean-soup-...,0,7,"[{'id': 6194, 'amount': 6.0, 'unit': 'cups', '...",0,[],[],NaN,NaN
1,False,False,True,False,False,False,True,False,False,19,no,20.0,10.0,1254,34,pinkwhen.com,pinkwhen.com,369.37,"[{'id': 10211821, 'aisle': 'Produce', 'image':...",715523,Chorizo and Beef Quinoa Stuffed Pepper,30,4,https://www.pinkwhen.com/chorizo-and-beef-quin...,https://img.spoonacular.com/recipes/715523-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",Chorizo and Beef Quinoa Stuffed Pepper is a <b...,[],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",96.067024,https://spoonacular.com/chorizo-and-beef-quino...,0,9,"[{'id': 10211821, 'amount': 4.0, 'unit': 'serv...",0,[],[],NaN,NaN
2,True,True,False,True,False,False,False,False,False,0,no,NaN,NaN,11,52,Foodista.com – The Cooking Encyclopedia Everyo...,Foodista,150.29,"[{'id': 18376, 'aisle': 'Bakery/Bread', 'image...",632252,Alouette® Stuffed Mushroom Caps,45,8,https://www.foodista.com/recipe/8LZ2BYQT/aloue...,https://img.spoonacular.com/recipes/632252-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",You can never have too many hor d'oeuvre recip...,[],"[antipasti, starter, snack, appetizer, antipas...","[dairy free, lacto ovo vegetarian, vegan]",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",96.040428,https://spoonacular.com/alouette-stuffed-mushr...,0,3,"[{'id': 18376, 'amount': 3.0, 'unit': 'tablesp...",0,[],[],CC BY 3.0,NaN
3,False,False,True,True,True,False,False,False,False,19,no,NaN,NaN,6,100,Foodista.com – The Cooking Encyclopedia Everyo...,Foodista,704.87,"[{'id': 1022009, 'aisle': 'Spices and Seasonin...",639891,Coffee-braised Short Ribs,45,5,https://www.foodista.com/recipe/8ZDDXLKG/coffe...,https://img.spoonacular.com/recipes/639891-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",Need a <b>gluten free and dairy free main cour...,[],"[lunch, main course, main dish, dinner]","[gluten free, dairy free]",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",96.030014,https://spoonacular.com/coffee-braised-short-r...,0,12,"[{'id': 1022009, 'amount': 1.5, 'unit': 'table...",0,[],[],CC BY 3.0,NaN
4,False,False,True,False,True,False,False,False,False,11,no,NaN,NaN,9,81,foodista.com,foodista.com,312.55,"[{'id': 11090, 'aisle': 'Produce', 'image': 'b...",646651,Herb chicken with sweet potato mash and sautée...,45,4,https://www.foodista.com/recipe/NHZKT2K8/serve...,https://img.spoonacular.com/recipes/646651-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",Herb chicken with sweet potato mash and sautée...,[],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",95.860924,https://spoonacular.com/herb-chicken-with-swee...,0,6,"[{'id': 11090, 'amount': 1.0, 'unit': 'mediu

In [35]:
df.iloc[24:28]

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,gaps,preparationMinutes,cookingMinutes,aggregateLikes,healthScore,creditsText,sourceName,pricePerServing,extendedIngredients,id,title,readyInMinutes,servings,sourceUrl,image,imageType,nutrition,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,usedIngredientCount,missedIngredientCount,missedIngredients,likes,usedIngredients,unusedIngredients,license,author
24,True,False,False,False,False,False,False,False,False,3,no,NaN,NaN,18,54,Foodista.com – The Cooking Encyclopedia Everyo...,Foodista,66.47,"[{'id': 10011090, 'aisle': 'Produce', 'image':...",636230,Broccoli with cheese soup,45,4,https://www.foodista.com/recipe/ZYJG37NG/brocc...,https://img.spoonacular.com/recipes/636230-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",Broccoli with cheese soup could be just the <b...,[],[side dish],[lacto ovo vegetarian],"[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",95.480194,https://spoonacular.com/broccoli-with-cheese-s...,0,5,"[{'id': 10011090, 'amount': 400.0, 'unit': 'g'...",0,[],[],CC BY 3.0,NaN
25,True,False,False,False,True,False,False,False,False,8,no,NaN,NaN,6,74,Foodista.com – The Cooking Encyclopedia Everyo...,Foodista,82.62,"[{'id': 10118375, 'aisle': 'Baking', 'image': ...",632347,Ancient Grains Bread,45,14,https://www.foodista.com/recipe/Y3ZZC68V/ancie...,https://img.spoonacular.com/recipes/632347-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",The recipe Ancient Grains Bread can be made <b...,[],[bread],[lacto ovo vegetarian],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",95.367393,https://spoonacular.com/ancient-grains-bread-6...,0,9,"[{'id': 10118375, 'amount': 1.0, 'unit': 'pack...",0,[],[],CC BY 3.0,NaN
26,True,True,True,True,True,False,False,False,False,11,no,NaN,NaN,0,100,blogspot.com,blogspot.com,155.62,"[{'id': 11090, 'aisle': 'Produce', 'image': 'b...",1132339,Garlic Lemon Chili Broccoli,30,2,https://foodwishes.blogspot.com/2010/06/will-t...,https://img.spoonacular.com/recipes/1132339-31...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...",If you want to add more <b>American</b> recipe...,[American],"[antipasti, soup, starter, snack, appetizer, a...","[gluten free, dairy free, paleolithic, lacto o...",[super bowl],"[{'name': '', 'steps': [{'number': 1, 'step': ...",95.336578,https://spoonacular.com/garlic-lemon-chili-bro...,0,6,"[{'id': 11090, 'amount': 1.5, 'unit': 'pounds'...",0,[],[],NaN,NaN
27,False,False,True,True,True,False,False,False,False,15,no,NaN,NaN,8,100,Foodista.com – The Cooking Encyclopedia Everyo...,Foodista,1157.26,"[{'id': 15270, 'aisle': 'Seafood', 'image': 's...",659927,Shrimp and Avocado Salad,45,4,https://www.foodista.com/recipe/ZTNWCWQP/shrim...,https://img.spoonacular.com/recipes/659927-312...,jpg,"{'nutrients': [{'name': 'Calories', 'amount': ...","Need a <b>gluten free, dairy free, and pescata...",[],"[side dish, lunch, main course, salad, main di...","[gluten free, dairy free, pescatarian]",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",95.263153,https://spoonacular.com/shrimp-and-avocado-sal...,0,13,"[{'id': 15270, 'amount': 4.0, 'unit': 'serving...",0,[],[],CC BY 3.0,NaN


## Dealing with recipes

In [36]:
# Define the required columns for the dataframe
dfrecipes_COLUMNS = [
    'vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy', 
    'cheap', 'veryPopular', 'sustainable', 'lowFodmap', 'pricePerServing', 
    'title', 'readyInMinutes', 'servings', 'sourceUrl', 'summary', 'license'
]

# Construct the dataframe
try:
    dfrecipes = pd.DataFrame(df, columns=dfrecipes_COLUMNS)
    main_logger.info(f"Constructed DataFrame with {len(dfrecipes)} recipes.")
    main_logger.debug(dfrecipes.head())
    main_logger.debug(dfrecipes.info())
except Exception as e:
    main_logger.error(f"Failed to construct DataFrame: {e}", exc_info=True)

# Rename columns for clarity and consistency
COLUMNS_TO_RENAME_MAP = {
    'vegetarian': 'is_vegetarian', 'vegan': 'is_vegan',
    'glutenFree': 'is_glutenFree', 'dairyFree': 'is_dairyFree',
    'veryHealthy': 'is_healthy', 'cheap': 'is_cheap',
    'veryPopular': 'is_Popular', 'sustainable': 'is_sustainable',
    'lowFodmap': 'is_lowFodmap', 'pricePerServing': 'price_per_serving', 
    'readyInMinutes': 'ready_min', 'sourceUrl': 'source_url', 
    'title': 'recipe_title'
}

try:
    dfrecipes = dfrecipes.rename(columns=COLUMNS_TO_RENAME_MAP)
    main_logger.info("Renamed columns.")
except Exception as e:
    main_logger.error(f"Failed to rename columns: {e}", exc_info=True)

# Extract a counter from which we start the id range
try:
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 0:
            counter_recipe = int(lines[0].strip())
        else:
            counter_recipe = 1
    main_logger.info(f"Counter recipe: {counter_recipe}")
except Exception as e:
    main_logger.error(f"Failed to extract counter_recipe: {e}", exc_info=True)

# Generate 'id_recipe' column
try:
    dfrecipes['id_recipe'] = range(counter_recipe, counter_recipe + len(dfrecipes))
    main_logger.info("Added 'id_recipe'.")
    main_logger.debug(dfrecipes.head())
    main_logger.debug(dfrecipes.info())
except Exception as e:
    main_logger.error(f"Failed to add 'id_recipe' column: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_recipe = counter_recipe + len(dfrecipes)
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_recipe) + "\n")
    print(f"'counter_recipe' appended to {file.name} successfully.")
except Exception as e:
    print(f"Error appending to file: {e}")

# Create a mapping dictionary from dfrecipes
try:
    RECIPE_MAPPING = dfrecipes.set_index('recipe_title')['id_recipe'].to_dict()
    main_logger.info("Created 'RECIPE_MAPPING'.")
except Exception as e:
    main_logger.error(f"Failed to create 'RECIPE_MAPPING': {e}", exc_info=True)



2024-07-20 18:35:45,956 - __main__ - INFO - Constructed DataFrame with 50 recipes.
2024-07-20 18:35:45,973 - __main__ - INFO - Renamed columns.
2024-07-20 18:35:45,975 - __main__ - INFO - Counter recipe: 51
2024-07-20 18:35:45,976 - __main__ - INFO - Added 'id_recipe'.
2024-07-20 18:35:45,990 - __main__ - INFO - Created 'RECIPE_MAPPING'.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   vegetarian       50 non-null     bool   
 1   vegan            50 non-null     bool   
 2   glutenFree       50 non-null     bool   
 3   dairyFree        50 non-null     bool   
 4   veryHealthy      50 non-null     bool   
 5   cheap            50 non-null     bool   
 6   veryPopular      50 non-null     bool   
 7   sustainable      50 non-null     bool   
 8   lowFodmap        50 non-null     bool   
 9   pricePerServing  50 non-null     float64
 10  title            50 non-null     object 
 11  readyInMinutes   50 non-null     int64  
 12  servings         50 non-null     int64  
 13  sourceUrl        50 non-null     object 
 14  summary          50 non-null     object 
 15  license          25 non-null     object 
dtypes: bool(9), float64(1), int64(2), object(4)
memory usage: 3.3+ K

In [37]:
dfrecipes.head()

,is_vegetarian,is_vegan,is_glutenFree,is_dairyFree,is_healthy,is_cheap,is_Popular,is_sustainable,is_lowFodmap,price_per_serving,recipe_title,ready_min,servings,source_url,summary,license,id_recipe
0,True,False,True,True,True,False,False,False,False,262.42,Smoky Black Bean Soup With Sweet Potato & Kale,630,6,https://www.foodista.com/recipe/B8SQD87X/smoky...,You can never have too many main course recipe...,NaN,51
1,False,False,True,False,False,False,True,False,False,369.37,Chorizo and Beef Quinoa Stuffed Pepper,30,4,https://www.pinkwhen.com/chorizo-and-beef-quin...,Chorizo and Beef Quinoa Stuffed Pepper is a <b...,NaN,52
2,True,True,False,True,False,False,False,False,False,150.29,Alouette® Stuffed Mushroom Caps,45,8,https://www.foodista.com/recipe/8LZ2BYQT/aloue...,You can never have too many hor d'oeuvre recip...,CC BY 3.0,53
3,False,False,True,True,True,False,False,False,False,704.87,Coffee-braised Short Ribs,45,5,https://www.foodista.com/recipe/8ZDDXLKG/coffe...,Need a <b>gluten free and dairy free main cour...,CC BY 3.0,54
4,False,False,True,False,True,False,False,False,False,312.55,Herb chicken with sweet potato mash and sautée...,45,4,https://www.foodista.com/recipe/NHZKT2K8/serve...,Herb chicken with sweet potato mash and sautée...,NaN,55


In [38]:
dfrecipes.iloc[24:28]

,is_vegetarian,is_vegan,is_glutenFree,is_dairyFree,is_healthy,is_cheap,is_Popular,is_sustainable,is_lowFodmap,price_per_serving,recipe_title,ready_min,servings,source_url,summary,license,id_recipe
24,True,False,False,False,False,False,False,False,False,66.47,Broccoli with cheese soup,45,4,https://www.foodista.com/recipe/ZYJG37NG/brocc...,Broccoli with cheese soup could be just the <b...,CC BY 3.0,75
25,True,False,False,False,True,False,False,False,False,82.62,Ancient Grains Bread,45,14,https://www.foodista.com/recipe/Y3ZZC68V/ancie...,The recipe Ancient Grains Bread can be made <b...,CC BY 3.0,76
26,True,True,True,True,True,False,False,False,False,155.62,Garlic Lemon Chili Broccoli,30,2,https://foodwishes.blogspot.com/2010/06/will-t...,If you want to add more <b>American</b> recipe...,NaN,77
27,False,False,True,True,True,False,False,False,False,1157.26,Shrimp and Avocado Salad,45,4,https://www.foodista.com/recipe/ZTNWCWQP/shrim...,"Need a <b>gluten free, dairy free, and pescata...",CC BY 3.0,78


## Dealing with ingredients

In [39]:
# 3.1 Reading Existing Ingredient Mapping
#----------------------------------------------------------------
# Function to read existing ING_MAPPING.txt
def read_ingredient_mapping(file_path):
    try:
        with open(file_path, 'r') as file:
            ing_mapping = json.load(file)
        return set(ing_mapping.keys()), ing_mapping
    except FileNotFoundError:
        main_logger.warning(f"{file_path} not found. Starting with an empty set.")
        return set(), {}
    except Exception as e:
        main_logger.error(f"Failed to read ING_MAPPING file: {e}", exc_info=True)
        return set(), {}

# Read existing ING_MAPPING.txt
existing_ingredients, existing_ingredients_mapping = read_ingredient_mapping('ING_MAPPING.txt')

#----------------------------------------------------------------
# 3.2 Extracting Ingredients from DataFrame
#----------------------------------------------------------------
# Define the required columns for the ingredients DataFrame
dfIng_COLUMNS = ['consistency', 'nameClean', 'aisle', 'name']

# Define a function to use it with the apply function
def extract_dicts(lst):
    """Extract dictionaries from a list of dictionaries and return a list"""
    return [d for d in lst if isinstance(d, dict)]

# Apply the function and sum the results
try:
    # 'all_ingredients' variable receives a list of all dictionaries in the extendedIngredients column
    all_ingredients = df['extendedIngredients'].apply(extract_dicts).sum()
    main_logger.info(f"Extracted {len(all_ingredients)} ingredients.")
except Exception as e:
    main_logger.error(f"Failed  extract ingredient dictionaries: {e}", exc_info=True)
    all_ingredients = []

#----------------------------------------------------------------
# 3.3 Ingredients DataFrame
#----------------------------------------------------------------
# Construct the ingredients DataFrame
try:
    dfIng = pd.DataFrame(all_ingredients, columns=dfIng_COLUMNS)
    main_logger.info(f"Constructed ingredients DataFrame with {len(dfIng)} ingredients.")
    main_logger.debug(dfIng.head())
    main_logger.debug(dfIng.info())
except Exception as e:
    main_logger.error(f"Failed to construct ingredients DataFrame: {e}", exc_info=True)
    dfIng = pd.DataFrame(columns=dfIng_COLUMNS)

# Fill missing 'nameClean' values with 'name' values
try:
    none_rows = dfIng[dfIng['nameClean'].isna()]
    main_logger.debug(none_rows)
    if not none_rows.empty:
        main_logger.warning(f"Found {len(none_rows)} null nameClean. Filling with name.")
        main_logger.debug(none_rows)
        dfIng['nameClean'] = dfIng['nameClean'].fillna(dfIng['name'])
        main_logger.info("Filled missing 'nameClean' values.")
    else:
        main_logger.info("No null 'nameClean' found.")
    main_logger.debug(dfIng.info())

    dfIng.drop('name', axis=1, inplace=True)
    main_logger.info("Dropped 'name' column.")
    main_logger.debug(dfIng.info())
except Exception as e:
    main_logger.error(f"Failed to handle null values: {e}", exc_info=True)

# Handle duplicates
try:
    duplicates = dfIng[dfIng.duplicated()]
    main_logger.debug(duplicates)
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate ingredients. Removing duplicates.")
        dfIng = dfIng.drop_duplicates().reset_index(drop=True)
        main_logger.info(f"DataFrame now contains {len(dfIng)} ingredients after removing duplicates.")
    else:
        main_logger.info("No duplicates found.")
    main_logger.debug(dfIng.info())
except Exception as e:
    main_logger.error(f"Failed to handle duplicate values: {e}", exc_info=True)

# Assign existing ingredient IDs and handle new ingredients
try:
    # Map existing ingredient IDs based on the nameClean column
    dfIng['id_ingredient'] = dfIng['nameClean'].map(existing_ingredients_mapping)

    # Identify new ingredients
    new_ingredients = dfIng[dfIng['id_ingredient'].isna()]

    # Extract a counter from which we start the id range
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 1:
            counter_ing = int(lines[1].strip())
            main_logger.info(f"Counter ingredients: {counter_ing}")
        else:
            counter_ing = 1
            main_logger.info(f"Counter ingredients initialized: {counter_ing}")

    # Assign new IDs to new ingredients
    new_ingredients['id_ingredient'] = range(counter_ing, counter_ing + len(new_ingredients))
    main_logger.info("New IDs assigned to new ingredients")

    # Combine existing and new ingredients
    dfIng.update(new_ingredients)
    main_logger.info("New ingredients IDs combined with existing ingredients.")

    # Convert id_ingredient to integer type where applicable, keeping None values
    dfIng['id_ingredient'] = dfIng['id_ingredient'].astype('Int64')
    main_logger.info("'id_ingredient' column converted to Integer")

except Exception as e:
    main_logger.error(f"Failed to assign IDs or combine ingredients: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_ing += len(new_ingredients)
    main_logger.info(f"Counter ingredients incremented: {counter_ing}")
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_ing) + "\n")
    main_logger.info(f"'counter_ing' appended to {file.name} successfully.")
except Exception as e:
    main_logger.error(f"Error appending to file: {e}", exc_info=True)

# Rename 'nameClean' to 'ing_name'
try:
    dfIng = dfIng.rename(columns={'nameClean': 'ing_name'})
    main_logger.info("Renamed 'nameClean' to 'ing_name'.")
    main_logger.debug(dfIng.info())
except Exception as e:
    main_logger.error(f"Failed to rename columns: {e}", exc_info=True)

# Create a mapping dictionary from dfIng
try:
    new_ingredients_mapping = dfIng.set_index('ing_name')['id_ingredient'].to_dict()
    main_logger.info("Created 'new_ingredients_mapping'.")
except Exception as e:
    main_logger.error(f"Failed to create 'new_ingredients_mapping': {e}", exc_info=True)

# Update ING_MAPPING with new ingredients
try:
    ING_MAPPING = {**existing_ingredients_mapping, **new_ingredients_mapping}
    with open('ING_MAPPING.txt', 'w') as file:
        json.dump(ING_MAPPING, file)
    main_logger.info("Updated and saved 'ING_MAPPING'.")
except Exception as e:
    main_logger.error(f"Failed to update and save 'ING_MAPPING': {e}", exc_info=True)


2024-07-20 18:35:58,296 - __main__ - INFO - Extracted 537 ingredients.
2024-07-20 18:35:58,299 - __main__ - INFO - Constructed ingredients DataFrame with 537 ingredients.
2024-07-20 18:35:58,312 - __main__ - WARNING - Found 4 null nameClean. Filling with name.
2024-07-20 18:35:58,315 - __main__ - INFO - Filled missing 'nameClean' values.
2024-07-20 18:35:58,326 - __main__ - INFO - Dropped 'name' column.
2024-07-20 18:35:58,339 - __main__ - WARNING - Found 285 duplicate ingredients. Removing duplicates.
2024-07-20 18:35:58,341 - __main__ - INFO - DataFrame now contains 252 ingredients after removing duplicates.
2024-07-20 18:35:58,348 - __main__ - INFO - Counter ingredients: 243
/tmp/ipykernel_9616/3613323008.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  537 non-null    object
 1   nameClean    533 non-null    object
 2   aisle        536 non-null    object
 3   name         537 non-null    object
dtypes: object(4)
memory usage: 16.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  537 non-null    object
 1   nameClean    537 non-null    object
 2   aisle        536 non-null    object
 3   name         537 non-null    object
dtypes: object(4)
memory usage: 16.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  537 non-null  

In [40]:
dfIng.head(30)

,consistency,ing_name,aisle,id_ingredient
0,LIQUID,chicken broth,Canned and Jarred,243
1,SOLID,chipotle chiles,Canned and Jarred,244
2,SOLID,dried black beans,Canned and Jarred,245
3,SOLID,ground cumin,Spices and Seasonings,151
4,SOLID,kale,Produce,20
5,LIQUID,olive oil,"Oil, Vinegar, Salad Dressing",7
6,SOLID,onion,Produce,16
7,SOLID,salt and pepper,Spices and Seasonings,10
8,SOLID,sweet potato,Produce,246
9,SOLID,bell pepper,Produce,72


## Dealing with measures (reference_ing)

In [41]:
# 4.1 Extract the column that contains all the ingredients measures along with the recipes titles
#----------------------------------------------------------------
try:
    dfrecipeIng = pd.DataFrame(df, columns=['extendedIngredients', 'title'])
    main_logger.info(f"Extracted ingredients and titles for {len(dfrecipeIng)} recipes.")
    main_logger.debug(dfrecipeIng.head())
    main_logger.debug(dfrecipeIng.info())
except Exception as e:
    logger.error("Failed to extract ingredients and titles", exc_info=True)

#----------------------------------------------------------------
# 4.2 Obtaining a new dataframe that contains measures of ingredients in each recipe
#----------------------------------------------------------------

# Define the required columns for the measures DataFrame
dfmeasures_COLUMNS =['nameClean', 'name', 'measures', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame
for i, row_series in dfrecipeIng.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'extendedIngredients' column for the current row
        recipe_ingredients_list = row_series['extendedIngredients']  # recipe_ingredients_list is a list
    
        # if recipe_ingredients_list is not an empty list skip the iteration (according to the Guard clauses principle of clean code)
        if not (recipe_ingredients_list and isinstance(recipe_ingredients_list, list)):
            continue
            
        # Iterate through each dictionary in the list
        for each_dict in recipe_ingredients_list:
            if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                # Get measures dict 
                measures_dict = each_dict.get('measures', {})
                # Transform measures dict to a list
                measures_list = [measures_dict]       
            else:
                measures_list = None
            
            # Create a dictionary for the row data
            row_data = {
                'nameClean': each_dict.get('nameClean', None),
                'measures': measures_list,
                'name': each_dict.get('name'),
                'title': title
            }
            
            # Append the row data to the list
            rows_data.append(row_data)
    except Exception as e:
        main_logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row_data
try:
    dfmeasures = pd.DataFrame(rows_data, columns=dfmeasures_COLUMNS)
    main_logger.info(f"Constructed measures DataFrame with {len(dfmeasures)} rows.")
    main_logger.debug(dfmeasures.head())
    main_logger.debug(dfmeasures.info())
except Exception as e:
    main_logger.error("Failed to construct measures DataFrame", exc_info=True)

# Handle missing values in the nameClean column
try:
    none_rows = dfmeasures[dfmeasures['nameClean'].isna()]
    main_logger.debug(none_rows)
    if not none_rows.empty:
        main_logger.warning(f"Found {len(none_rows)} null nameClean. Filling with name.")
        dfmeasures['nameClean'] = dfmeasures['nameClean'].fillna(dfmeasures['name'])
        main_logger.info("Filled missing 'nameClean' values.")
    else:
        main_logger.info("No null 'nameClean' found.")
except Exception as e:
    main_logger.error("Failed to handle null values in 'nameClean'", exc_info=True)

# Drop the 'name' column after filling 'nameClean'
try:
    dfmeasures.drop('name', axis=1, inplace=True)
    main_logger.info("Dropped 'name' column.")
except Exception as e:
    main_logger.error("Failed to drop 'name' column", exc_info=True)

# Rename 'nameClean' to 'ing_name'
try:
    dfmeasures = dfmeasures.rename(columns={'nameClean': 'ing_name'})
    main_logger.info("Renamed 'nameClean' to 'ing_name'.")
    main_logger.debug(dfmeasures.info())
except Exception as e:
    main_logger.error("Failed to rename columns", exc_info=True)

#----------------------------------------------------------------
# 4.3 transforming the measures column
#----------------------------------------------------------------

# Function to extract measure information from measures dict
def extract_measure(measures_dict, key_name):
  """
  Extracts a string value representing the measure from the given dictionary using a given name.
  Handles cases where the key name might be different.

  Args:
      measures_dict: A dictionary containing the measure information.
      key_name: The key name to look for.

  Returns:
      A string representing the measure in the format "amount unitShort".
  """
  if key_name in measures_dict:
    return f"{measures_dict[key_name]['amount']} {measures_dict[key_name]['unitShort']}"
  else:
    # Handle cases where the key might be different
    for key in measures_dict:
      if isinstance(measures_dict[key], dict):
        return extract_measure(measures_dict[key], key)
    # If no matching key is found, return an empty string
    return None

# Create two new columns with extracted measures
try:
    dfmeasures['measure_1'] = dfmeasures['measures'].apply(lambda x: extract_measure(x[0], "us"))
    dfmeasures['measure_2'] = dfmeasures['measures'].apply(lambda x: extract_measure(x[0], "metric"))
    main_logger.info("Created columns 'measure_1' and 'measure_2'.")
except Exception as e:
    logger.error("Failed to create 'measure_1' and 'measure_2' columns", exc_info=True)

# Drop the original 'measures' column
try:
    dfmeasures.drop('measures', axis=1, inplace=True)
    main_logger.info("Dropped 'measures' column.")
    main_logger.debug(dfmeasures.head())
    main_logger.debug(dfmeasures.info())
except Exception as e:
    main_logger.error("Failed to drop 'measures' column", exc_info=True)

# Function to combine measures from 'measure_1' and 'measure_2'
def combine_measures(row):
  """
  Combines values from 'measure_1' and 'measure_2' columns into a single string.

  Args:
      row: A pandas Series representing a row of the DataFrame.

  Returns:
      A string containing the combined measure value(s).
  """
  measure_1 = row['measure_1']
  measure_2 = row['measure_2']

  if measure_1 == measure_2:
    return measure_1  # Same values, return one
  else:
    return f"{measure_1} / {measure_2}"  # Different values are concatenated with "/"

# Apply the function to create a new 'measure' column
try:
    dfmeasures['measure'] = dfmeasures.apply(combine_measures, axis=1)
    main_logger.info("Created 'measure' column by combining 'measure_1' and 'measure_2'.")
except Exception as e:
    main_logger.error("Failed to create 'measure' column", exc_info=True)

# Drop 'measure_1' and 'measure_2' columns
try:
    dfmeasures.drop(['measure_1', 'measure_2'], axis=1, inplace=True)
    main_logger.info("Dropped 'measure_1' and 'measure_2' columns.")
    main_logger.debug(dfmeasures.head())
    main_logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to drop 'measure_1' and 'measure_2' columns", exc_info=True)

#----------------------------------------------------------------
# 4.4 dealing with id_recipe in dfmeasures
#----------------------------------------------------------------

# Function to map recipe titles to id_recipe
def map_with_none(recipe_title, RECIPE_MAPPING):
    """Maps recipe names to id_recipe, handling missing values."""
    if pd.isna(recipe_title):
        return None  # Return None for missing titles
    return RECIPE_MAPPING.get(recipe_title, None)  # Use get() to avoid KeyError for missing keys

try:
    # Apply the mapping function to create 'id_recipe' column
    dfmeasures['id_recipe'] = dfmeasures['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    main_logger.info("Added 'id_recipe' column based on 'RECIPE_MAPPING'.")
    # Convert id_recipe to integer type where applicable, keeping None values
    dfmeasures['id_recipe'] = dfmeasures['id_recipe'].astype('Int64')
    main_logger.info("Converted 'id_recipe' column to Integer type.")
    main_logger.debug(dfmeasures.head())
    main_logger.debug(dfmeasures.info())
except Exception as e:
    main_logger.error("Failed to add or convert 'id_recipe' column", exc_info=True)

#----------------------------------------------------------------
# 4.5 dealing with id_ingredient in dfmeasures
#----------------------------------------------------------------

# Function to map ingredient names to id_ingredient
def map_with_none(ing_name, ING_MAPPING):
    """Maps ingredient names to id_ingredient, handling missing values."""
    if pd.isna(ing_name):
        return None  # Return None for missing titles
    return ING_MAPPING.get(ing_name, None)  # Use get() to avoid KeyError for missing keys
    
try:
    # Apply the mapping function to create 'id_ingredient' column
    dfmeasures['id_ingredient'] = dfmeasures['ing_name'].apply(map_with_none, args=(ING_MAPPING,))
    main_logger.info("Added 'id_ingredient' column based on 'ING_MAPPING'.")
    # Convert id_ingredient to integer type where applicable, keeping None values
    dfmeasures['id_ingredient'] = dfmeasures['id_ingredient'].astype('Int64')
    main_logger.info("Converted 'id_ingredient' column to Integer type.")
    main_logger.debug(dfmeasures.head())
    main_logger.debug(dfmeasures.info())
except Exception as e:
    main_logger.error(f"Failed to add or convert 'id_ingredient' column: {e}", exc_info=True)

#----------------------------------------------------------------
# 4.6 Some Cleaning
#----------------------------------------------------------------

# Drop 'ing_name' and 'title' columns to finalize the reference DataFrame
try:
    dfreference_ing = dfmeasures.drop(['ing_name', 'title'], axis=1)
    main_logger.info("Dropped 'ing_name' and 'title' columns.")
    main_logger.debug(dfreference_ing.head())
    main_logger.debug(dfreference_ing.info())
except Exception as e:
    logger.error("Failed to finalize reference DataFrame by dropping 'ing_name' and 'title' columns", exc_info=True)


try:
    # Find duplicates
    duplicates = dfreference_ing[dfreference_ing.duplicated()]
    main_logger.debug(duplicates)
    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate reference ingredients. Removing duplicates.")
        dfreference_ing = dfreference_ing.drop_duplicates().reset_index(drop=True)
        main_logger.info(f"DataFrame now contains {len(dfreference_ing)} reference ingredients after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    main_logger.error("Failed to handle duplicates", exc_info=True)


2024-07-20 18:36:03,904 - __main__ - INFO - Extracted ingredients and titles for 50 recipes.
2024-07-20 18:36:03,936 - __main__ - INFO - Constructed measures DataFrame with 537 rows.
2024-07-20 18:36:03,956 - __main__ - WARNING - Found 4 null nameClean. Filling with name.
2024-07-20 18:36:03,958 - __main__ - INFO - Filled missing 'nameClean' values.
2024-07-20 18:36:03,963 - __main__ - INFO - Dropped 'name' column.
2024-07-20 18:36:03,967 - __main__ - INFO - Renamed 'nameClean' to 'ing_name'.
2024-07-20 18:36:03,974 - __main__ - INFO - Created columns 'measure_1' and 'measure_2'.
2024-07-20 18:36:03,976 - __main__ - INFO - Dropped 'measures' column.
2024-07-20 18:36:03,997 - __main__ - INFO - Created 'measure' column by combining 'measure_1' and 'measure_2'.
2024-07-20 18:36:03,998 - __main__ - INFO - Dropped 'measure_1' and 'measure_2' columns.
2024-07-20 18:36:04,011 - __main__ - INFO - Added 'id_recipe' column based on 'RECIPE_MAPPING'.
2024-07-20 18:36:04,013 - __main__ - INFO - Co

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   extendedIngredients  50 non-null     object
 1   title                50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nameClean  533 non-null    object
 1   name       537 non-null    object
 2   measures   537 non-null    object
 3   title      537 non-null    object
dtypes: object(4)
memory usage: 16.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ing_name  537 non-null    object
 1   measures  537 non-null    object
 2   title     537 non

In [42]:
dfreference_ing.head(30)

,measure,id_recipe,id_ingredient
0,6.0 cups / 1.41 l,51,243
1,3.0 Tbsps,51,244
2,1.0 lb / 453.592 g,51,245
3,2.0 Tbsps,51,151
4,1.0 bunch,51,20
5,2.0 Tbsps,51,7
6,1.0,51,16
7,6.0 servings,51,10
8,3.0 lb / 1.361 kgs,51,246
9,4.0 servings,52,72


## Dealing with steps

In [43]:
# 5.1 Extract the column that contains all the instructions along with the recipes titles
#----------------------------------------------------------------
try:
    dfAllIns = pd.DataFrame(df, columns=['analyzedInstructions', 'title'])
    main_logger.info(f"Extracted instructions for {len(dfAllIns)} recipes.")
    main_logger.debug(dfAllIns.head())
    main_logger.debug(dfAllIns.info())
except Exception as e:
    main_logger.error(f"Failed to extract instructions and titles: {e}", exc_info=True)

#---------------------------------------------------------
# 5.2 obtaining a new dataframe that contains steps of each recipe
#---------------------------------------------------------

# Define the required columns for the steps DataFrame
dfsteps_COLUMNS = ['steps', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Define the required columns for the steps DataFrame
for i, row_series in dfAllIns.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'analyzedInstructions' column for the current row
        instructions_list = row_series['analyzedInstructions']  # instructions_list is a list containing one dictionary
        
        if instructions_list and isinstance(instructions_list[0], dict):  # Check if instructions_list is non-empty and its first element is a dictionary
            # Get steps list
            steps_list = instructions_list[0].get('steps', [])
        else:
            steps_list = None
        
        # Create a dictionary for the row data
        row_data = {
            'steps': steps_list,
            'title': title
        }
        
        # Append the row data to the list
        rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}': {e}", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfsteps = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    main_logger.info(f"Constructed steps DataFrame for {len(dfsteps)} recipes.")
    main_logger.debug(dfsteps.head())
    main_logger.debug(dfsteps.info())
except Exception as e:
    main_logger.error(f"Failed to construct steps DataFrame: {e}", exc_info=True)


#---------------------------------------------------------
# 5.3 transforming the steps column
#---------------------------------------------------------

# Define the required columns for the step DataFrame
dfsteps_COLUMNS = ['equipment', 'step', 'length', 'number', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame dfsteps
for i, row_series in dfsteps.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'steps' column for the current row
        steps_list = row_series['steps']
        
        if steps_list is not None:
            # Iterate through each dictionary in the steps_list
            for each_dict in steps_list:
                if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                    # Extract relevant information from each_dict
                    number = each_dict.get('number', None)
                    step = each_dict.get('step', None)
                    time = each_dict.get('length', {}).get('number', None)
                    unit = each_dict.get('length', {}).get('unit', None)
                    
                    # Calculate length based on time and unit
                    if time is None or unit is None:
                        length = None
                    else:
                        length = f"{time} {unit}"
                    
                    equipment_list = each_dict.get('equipment', [])  # Get equipment list or empty list if 'equipment' is missing
                    
                    # Handle NaN or None values in equipment_list
                    if isinstance(equipment_list, list):
                        equipment_list = [e if pd.notna(e) else None for e in equipment_list]
                    
                    # Create a dictionary for the row data
                    row_data = {
                        'length': length,
                        'number': number,
                        'step': step,
                        'equipment': equipment_list,
                        'title': title
                    }
                    
                    # Append the row data to the list
                    rows_data.append(row_data)
        else:
            # If steps_list is None, create a row with None values
            row_data = {
                'length': None,
                'number': None,
                'step': None,
                'equipment': None,
                'title': title
            }
            
            # Append the row data to the list
            rows_data.append(row_data)
    except Exception as e:
        main_logger.error(f"Error processing row {i} for recipe '{title}': {e}", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfstep = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    main_logger.info(f"Constructed steps DataFrame with {len(dfstep)} steps.")
    main_logger.debug(dfstep.head())
    main_logger.debug(dfstep.info())
except Exception as e:
    main_logger.error(f"Failed to construct steps DataFrame: {e}", exc_info=True)

# Find duplicates
try:
    duplicates = dfstep[dfstep.duplicated(subset=["step", "length", "number", "title"])]
    main_logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate steps. Removing duplicates.")
        dfstep = dfstep.drop_duplicates()
        dfstep.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfstep)} steps after removing duplicates.")
    else:
        main_logger.info("No duplicates found")
except Exception as e:
    main_logger.error(f"Failed to find or remove duplicates: {e}", exc_info=True)

# Checking if there's a Null values in step column
try:
    none_rows = dfstep[dfstep['step'].isna()]
    main_logger.debug(none_rows)

    # Deleting the null steps in dfstep DataFrame.
    if len(none_rows) > 0:
        main_logger.warning(f"Found {len(none_rows)} null steps. Deleting null steps.")
        dfstep.dropna(subset=['step'], inplace=True)
        dfstep.reset_index(drop=True, inplace=True)
        main_logger.info("Null steps deleted")
        main_logger.info(f"DataFrame now contains {len(dfstep)} steps after removing null values.")
    else:
        main_logger.info("No null steps found")
except Exception as e:
    main_logger.error(f"Failed to handle null steps: {e}", exc_info=True)

main_logger.debug(dfstep.head())
main_logger.debug(dfstep.info())   
    
#---------------------------------------------------------
# 5.4 transforming equipment column
#---------------------------------------------------------

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame dfstep
for i, row_series in dfstep.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']
        # Access the step length for the current row 
        length = row_series['length']
        # Access the recipe step for the current row 
        step = row_series['step']
        # Access the step number for the current row 
        number = int(row_series['number'])
        # Access the 'equipment' column for the current row
        equipments_list = row_series['equipment']  
        
        # Initialize variables to store processed data
        equipments_name_list = []
        
        # Process equipments_list
        if equipments_list and isinstance(equipments_list, list):
            for each_dict in equipments_list:
                if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                    # Get a list containing equipments names
                    equipment_name = each_dict.get('name', None)
                    if equipment_name is not None:  # Ensure equipment_name is not None
                        equipments_name_list.append(equipment_name)
        else:
            equipments_name_list = None  # Handle case where equipments_list is None or not a list
        
        # Create a dictionary for the row data
        row_data = {
            'length': length,
            'number': number,
            'step': step,
            'equipment': equipments_name_list,
            'title': title
        }
        
        # Append the row data to the list
        rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfstepclean = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    main_logger.info(f"Constructed dfstepclean DataFrame with {len(dfstepclean)} steps and contains cleaned lists of equipments")
    main_logger.debug(dfstepclean.head())
    main_logger.debug(dfstepclean.info())
except Exception as e:
    main_logger.error(f"Failed to construct dfstepclean DataFrame: {e}", exc_info=True)

#---------------------------------------------------------
# 5.6 dealing with dfstep_final
#---------------------------------------------------------
# Extract a counter from which we start the id range
try:
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 2:
            counter_step = int(lines[2].strip())
        else:
            counter_step = 1
    main_logger.info(f"Counter step: {counter_step}")
except Exception as e:
    main_logger.error(f"Failed to extract counter_step: {e}", exc_info=True)

# Generate 'id_step' column
try:
    dfstepclean['id_step'] = range(counter_step, counter_step + len(dfstepclean))
    main_logger.info("Added 'id_step'.")
    main_logger.debug(dfstepclean.head())
    main_logger.debug(dfstepclean.info())
except Exception as e:
    main_logger.error(f"Failed to add 'id_recipe' column: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_step = counter_step + len(dfstepclean)
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_step) + "\n")
    print(f"'counter_step' appended to {file.name} successfully.")
except Exception as e:
    print(f"Error appending to file: {e}")


# add id_recipe column 
try:
    dfstep_final = dfstepclean.copy()
    main_logger.debug(dfstep_final.info())

    # Function to map recipe titles to id_recipe
    def map_with_none(recipe_title, RECIPE_MAPPING):
        """Maps recipe names to id_recipe, handling missing values."""
        if pd.isna(recipe_title):
            return None  # Return None for missing titles
        return RECIPE_MAPPING.get(recipe_title, None)  # Use get() to avoid KeyError for missing keys

    # Apply the mapping function to create 'id_recipe' column
    dfstep_final['id_recipe'] = dfstep_final['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    main_logger.info("Added 'id_recipe' column based on 'RECIPE_MAPPING'.")
    # Convert id_recipe to integer type where applicable, keeping None values
    dfstep_final['id_recipe'] = dfstep_final['id_recipe'].astype('Int64')
    main_logger.info("Converted 'id_recipe' column to Integer type.")
    main_logger.debug(dfstep_final.head())
    main_logger.debug(dfstep_final.info())

    # Drop columns by label (column name)
    dfstep_final.drop(columns=['equipment', 'title'], inplace=True)
    main_logger.info("'equipment' and 'title' columns deleted")
    main_logger.debug(dfstep_final.head())
    main_logger.debug(dfstep_final.info())
except Exception as e:
    main_logger.error(f"Failed to finalize dfstep_final DataFrame: {e}", exc_info=True)




2024-07-20 18:36:08,885 - __main__ - INFO - Extracted instructions for 50 recipes.
2024-07-20 18:36:08,899 - __main__ - INFO - Constructed steps DataFrame for 50 recipes.
2024-07-20 18:36:08,930 - __main__ - INFO - Constructed steps DataFrame with 303 steps.
2024-07-20 18:36:08,939 - __main__ - INFO - No duplicates found
2024-07-20 18:36:08,944 - __main__ - WARNING - Found 1 null steps. Deleting null steps.
2024-07-20 18:36:08,946 - __main__ - INFO - Null steps deleted
2024-07-20 18:36:08,948 - __main__ - INFO - DataFrame now contains 302 steps after removing null values.
2024-07-20 18:36:08,994 - __main__ - INFO - Constructed dfstepclean DataFrame with 302 steps and contains cleaned lists of equipments
2024-07-20 18:36:09,011 - __main__ - INFO - Counter step: 276
2024-07-20 18:36:09,012 - __main__ - INFO - Added 'id_step'.
2024-07-20 18:36:09,031 - __main__ - INFO - Added 'id_recipe' column based on 'RECIPE_MAPPING'.
2024-07-20 18:36:09,033 - __main__ - INFO - Converted 'id_recipe' co

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   analyzedInstructions  50 non-null     object
 1   title                 50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   steps   49 non-null     object
 1   title   50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   equipment  302 non-null    object 
 1   step       302 non-null    object 
 2   length     78 non-null     object 
 3   number     302 non-null    float64
 4   title      30

In [44]:
dfstep_final.head()

,step,length,number,id_step,id_recipe
0,Spread the dry beans out on a baking sheet and...,None,1,276,51
1,"Place the beans in a big soup pot, cover with ...",600 minutes,2,277,51
2,Drain the beans.,None,3,278,51
3,Heat the oil in a soup pot over med-high heat.,None,4,279,51
4,Add the onion along with a pinch of salt and s...,30 minutes,5,280,51


In [45]:
dfstep_final.iloc[130:140]

,step,length,number,id_step,id_recipe
130,Tie the rolled steak with twine at 2-3 inch in...,None,8,406,72
131,Season the outside with salt and pepper.,None,9,407,72
132,Heat the remaining 2 tablespoons olive oil in ...,2 minutes,10,408,72
133,Place in the oven and roast for 20 to 25 minutes.,20 minutes,11,409,72
134,"Transfer to a cutting board, tend with aluminu...",10 minutes,12,410,72
135,"To serve, remove the twine from the roast and ...",None,13,411,72
136,Peel potatoes and put them in a large pot fill...,None,1,412,73
137,"Chop the mushrooms, carrot and red bell pepper.",None,2,413,73
138,Add them into your food processor and pulse un...,None,3,414,73
139,Heat oil in a large skillet.,None,4,415,73


## Dealing with instructions

In [46]:
# Extract a counter from which we start the id range

try:
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 3:
            counter_inst = int(lines[3].strip())
        else:
            counter_inst = 1
    main_logger.info(f"Counter instruction: {counter_inst}")
except Exception as e:
    main_logger.error(f"Failed to extract 'counter_inst': {e}", exc_info=True)



try:
    # Assign a unique number to each unique title
    dfstep_final['instruction_id'] = pd.factorize(dfstep_final['id_recipe'])[0] + counter_inst
    main_logger.info("Added 'instrction_id'.")
    main_logger.debug(dfstep_final.head(30))
    main_logger.debug(dfstep_final.info())
except Exception as e:
    main_logger.error(f"Failed to add 'instruction_id': {e}", exc_info=True)


try:
    # Create the instructions dataframe
    dfIns = dfstep_final[['instruction_id', 'id_recipe']]
    main_logger.info("Constructed instructions dataframe")
    main_logger.debug(dfIns.head())
    main_logger.debug(dfIns.info())
except Exception as e:
    main_logger.error(f"Failed to construct instructions dataframe: {e}", exc_info=True)

try:
    # Find duplicates
    duplicates = dfIns[dfIns.duplicated()]
    main_logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate instructions. Removing duplicates.")
        dfIns = dfIns.drop_duplicates()
        dfIns.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfIns)} instructions after removing duplicates.")
    else:
        main_logger.info("No duplicates found")
except Exception as e:
    main_logger.error("An unexpected error occurred while handling duplicates.", exc_info=True)


# Append the counter content to the file
try:
    counter_inst = counter_inst + len(dfIns)
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_inst) + "\n")
    main_logger.info(f"'counter_inst' appended to {file.name} successfully.")
except Exception as e:
    main_logger.error(f"Error appending to file: {e}")

#Drop columns
try:
    dfstep_final.drop(columns=['id_recipe'], inplace=True)
    main_logger.info("'id_recipe' deleted from steps dataframe")
except Exception as e:
    main_logger.error("Failed to drop 'id_recipe' column.", exc_info=True)

main_logger.debug(dfIns.head())
main_logger.debug(dfIns.info())
main_logger.debug(dfstep_final.info())



2024-07-20 18:36:21,801 - __main__ - INFO - Counter instruction: 50
2024-07-20 18:36:21,803 - __main__ - INFO - Added 'instrction_id'.
2024-07-20 18:36:21,812 - __main__ - INFO - Constructed instructions dataframe
2024-07-20 18:36:21,821 - __main__ - WARNING - Found 253 duplicate instructions. Removing duplicates.
2024-07-20 18:36:21,823 - __main__ - INFO - DataFrame now contains 49 instructions after removing duplicates.
2024-07-20 18:36:21,824 - __main__ - INFO - 'counter_inst' appended to new_counters.txt successfully.
2024-07-20 18:36:21,825 - __main__ - INFO - 'id_recipe' deleted from steps dataframe


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   step            302 non-null    object
 1   length          78 non-null     object
 2   number          302 non-null    int64 
 3   id_step         302 non-null    int64 
 4   id_recipe       302 non-null    Int64 
 5   instruction_id  302 non-null    int64 
dtypes: Int64(1), int64(3), object(2)
memory usage: 14.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   instruction_id  302 non-null    int64
 1   id_recipe       302 non-null    Int64
dtypes: Int64(1), int64(1)
memory usage: 5.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------   

In [47]:
dfIns.head(50)

,instruction_id,id_recipe
0,50,51
1,51,52
2,52,53
3,53,54
4,54,55
5,55,56
6,56,57
7,57,58
8,58,60
9,59,61


In [48]:
dfstep_final.head()

,step,length,number,id_step,instruction_id
0,Spread the dry beans out on a baking sheet and...,None,1,276,50
1,"Place the beans in a big soup pot, cover with ...",600 minutes,2,277,50
2,Drain the beans.,None,3,278,50
3,Heat the oil in a soup pot over med-high heat.,None,4,279,50
4,Add the onion along with a pinch of salt and s...,30 minutes,5,280,50


## Dealing with equipments

In [49]:
# 7.1 Extract Equipments names
#---------------------------------------------------------
# Define the required columns for the equipments DataFrame
dfequip_COLUMNS = ['name']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame
for i, row_series in dfstep.iterrows():
    try:
        equip_list = row_series['equipment']
        
        if equip_list and isinstance(equip_list, list):
            for each_dict in equip_list:
                # Check if the element is a dictionary
                if each_dict and isinstance(each_dict, dict):
                    name = each_dict.get('name', None)
                    
                    # Create a dictionary for the row data
                    row_data = {
                        'name': name
                    }
                    
                    # Append the row data to the list
                    rows_data.append(row_data)
                else:
                    main_logger.warning(f"Non-dictionary element found in equipment list at row {i}. Skipping element.")
        else:
            main_logger.warning(f"Equipment list is empty or not a list at row {i}. Skipping row.")

    except Exception as e:
        main_logger.error(f"Error processing row {i}", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfequip = pd.DataFrame(rows_data, columns=dfequip_COLUMNS)
    main_logger.info(f"Constructed equipments DataFrame with {len(dfequip)} equipments.")
    main_logger.debug(dfequip.head())
    main_logger.debug(dfequip.info())
except Exception as e:
    main_logger.error("Failed to construct equipments DataFrame.", exc_info=True)
    dfequip = pd.DataFrame(columns=dfequip_COLUMNS)  # Create an empty DataFrame as a fallback

try:
    # Find duplicates
    duplicates = dfequip[dfequip.duplicated()]
    main_logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate equipments. Removing duplicates.")
        dfequip = dfequip.drop_duplicates()
        dfequip.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfequip)} equipments after removing duplicates.")
    else:
        main_logger.info("No duplicates found")
except Exception as e:
    main_logger.error("An error occurred while finding or removing duplicates.", exc_info=True)

# Rename 'name' column to 'equip_name'
try:
    dfequip = dfequip.rename(columns={'name': 'equip_name'})
    main_logger.info("Renamed column 'name' to 'equip_name'.")
    main_logger.debug(dfequip.head())
    main_logger.debug(dfequip.info())
except KeyError as e:
    main_logger.error(f"Failed to rename 'name' column because it is missing: {e}", exc_info=True)
except Exception as e:
    main_logger.error(f"An unexpected error occurred while renaming 'name' column: {e}", exc_info=True)
#----------------------------------------------------------------
# 7.2 Reading Existing Equipment Mapping
#----------------------------------------------------------------
# Function to read existing ING_MAPPING.txt
def read_equipment_mapping(file_path):
    try:
        with open(file_path, 'r') as file:
            EQUIP_MAPPING = json.load(file)
        return set(EQUIP_MAPPING.keys()), EQUIP_MAPPING
    except FileNotFoundError:
        main_logger.warning(f"{file_path} not found. Starting with an empty set.")
        return set(), {}
    except Exception as e:
        main_logger.error(f"Failed to read EQUIP_MAPPING file: {e}", exc_info=True)
        return set(), {}

# Read existing EQUIP_MAPPING.txt
existing_equipments, existing_equipments_mapping = read_equipment_mapping('EQUIP_MAPPING.txt')
main_logger.info("mapping read")
#----------------------------------------------------------------
# 7.3 Construct the id_equipment column
#----------------------------------------------------------------

# Assign existing equipment IDs and handle new equipments
try:
    # Map existing equipment IDs based on the equip_name column
    dfequip['id_equipment'] = dfequip['equip_name'].map(existing_equipments_mapping)

    # Identify new equipments
    new_equipments = dfequip[dfequip['id_equipment'].isna()]

    # Extract a counter from which we start the id range
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 4:
            counter_equip = int(lines[4].strip())
            main_logger.info(f"Counter equipments: {counter_equip}")
        else:
            counter_equip = 1
            main_logger.info(f"Counter equipments initialized: {counter_equip}")

    # Assign new IDs to new counter_equip
    new_equipments['id_equipment'] = range(counter_equip, counter_equip + len(new_equipments))
    main_logger.info("New IDs assigned to new equipments")

    # Combine existing and new equipments
    dfequip.update(new_equipments)
    main_logger.info("New equipments IDs combined with existing equipments.")

    # Convert id_equipment to integer type 
    dfequip['id_equipment'] = dfequip['id_equipment'].astype('Int64')
    main_logger.info("'id_equipment' column converted to Integer")

except Exception as e:
    main_logger.error(f"Failed to assign IDs or combine equipments: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_equip += len(new_equipments)
    main_logger.info(f"Counter equipments incremented: {counter_equip}")
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_equip) + "\n")
    main_logger.info(f"'counter_equip' appended to {file.name} successfully.")
except Exception as e:
    main_logger.error(f"Error appending to file: {e}", exc_info=True)


# Create a mapping dictionary from dfequip
try:
    new_equipments_mapping = dfequip.set_index('equip_name')['id_equipment'].to_dict()
    main_logger.info("Created 'new_equipments_mapping'.")
except Exception as e:
    main_logger.error(f"Failed to create 'new_equipments_mapping': {e}", exc_info=True)

# Update EQUIP_MAPPING with new equipments
try:
    EQUIP_MAPPING = {**existing_equipments_mapping, **new_equipments_mapping}
    with open('EQUIP_MAPPING.txt', 'w') as file:
        json.dump(EQUIP_MAPPING, file)
    main_logger.info("Updated and saved 'EQUIP_MAPPING'.")
except Exception as e:
    main_logger.error(f"Failed to update and save 'EQUIP_MAPPING': {e}", exc_info=True)



2024-07-20 18:36:31,498 - __main__ - WARNING - Equipment list is empty or not a list at row 2. Skipping row.
2024-07-20 18:36:31,500 - __main__ - WARNING - Equipment list is empty or not a list at row 4. Skipping row.
2024-07-20 18:36:31,501 - __main__ - WARNING - Equipment list is empty or not a list at row 5. Skipping row.
2024-07-20 18:36:31,502 - __main__ - WARNING - Equipment list is empty or not a list at row 8. Skipping row.
2024-07-20 18:36:31,504 - __main__ - WARNING - Equipment list is empty or not a list at row 11. Skipping row.
2024-07-20 18:36:31,505 - __main__ - WARNING - Equipment list is empty or not a list at row 12. Skipping row.
2024-07-20 18:36:31,506 - __main__ - WARNING - Equipment list is empty or not a list at row 13. Skipping row.
2024-07-20 18:36:31,507 - __main__ - WARNING - Equipment list is empty or not a list at row 15. Skipping row.
2024-07-20 18:36:31,509 - __main__ - WARNING - Equipment list is empty or not a list at row 18. Skipping row.
2024-07-20 18:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    172 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   equip_name  31 non-null     object
dtypes: object(1)
memory usage: 376.0+ bytes


In [50]:
dfequip.head(10)

,equip_name,id_equipment
0,baking sheet,23
1,pot,3
2,blender,19
3,frying pan,9
4,oven,14
5,mixing bowl,10
6,grill,15
7,bowl,6
8,baking pan,24
9,cutting board,30


## Dealing with reference_equip

In [51]:
# 8.1 'equip_name' column
#----------------------------------------------------------------

# Explode the equipment column in dfstepclean dataframe because equipment is a list of equipments 
try:
    dfstepclean_exploded = dfstepclean.explode(['equipment'])
    dfstepclean_exploded = dfstepclean_exploded.reset_index(drop=True)
    main_logger.info("'equipment' column in step dataframe exploded.")
except Exception as e:
    main_logger.error("Failed to explode 'equipment' column.", exc_info=True)

# Rename the exploded column to equip_name
try:
    dfstepclean_exploded = dfstepclean_exploded.rename(columns={'equipment': 'equip_name'})
    main_logger.info("Renamed column 'equipment' to 'equip_name'.")
    main_logger.debug(dfstepclean_exploded.head())
except Exception as e:
    main_logger.error("Failed to rename 'equipment' column", exc_info=True)

#----------------------------------------------------------------
# 8.2 'id_equipment' column
#----------------------------------------------------------------

# Function to map equipment names to id_equipment, handling missing values
def map_with_none(equip_name, EQUIP_MAPPING):
    """Maps equipment names to id_equipment, handling missing values."""
    if pd.isna(equip_name):
        return None  # Return None for missing equipment names
    return EQUIP_MAPPING.get(equip_name, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to fill the new column
try:
    dfstepclean_exploded['id_equipment'] = dfstepclean_exploded['equip_name'].apply(map_with_none, args=(EQUIP_MAPPING,))
    main_logger.info("'id_equipment' column added according to the 'EQUIP_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to add 'id_equipment' column.", exc_info=True)

# Convert id_equipment to integer type where applicable, keeping None values
try:
    dfstepclean_exploded['id_equipment'] = dfstepclean_exploded['id_equipment'].astype('Int64')
    main_logger.info("'id_equipment' column converted to Integer.")
    main_logger.debug(dfstepclean_exploded.head())
    main_logger.debug(dfstepclean_exploded.info())
except Exception as e:
    main_logger.error("Failed to convert 'id_equipment' column to Integer.", exc_info=True)

#----------------------------------------------------------------
# 8.3 'id_recipe' column
#----------------------------------------------------------------

# dealing with id_recipe in dfstepclean_exploded
try:
    # Apply the custom map function to fill the new column
    dfstepclean_exploded['id_recipe'] = dfstepclean_exploded['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    main_logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to add 'id_recipe' column.", exc_info=True)
    
try:
    # Convert id_recipe to integer type where applicable, keeping None values
    dfstepclean_exploded['id_recipe'] = dfstepclean_exploded['id_recipe'].astype('Int64')
    main_logger.info("'id_recipe' column converted to Integer.")
    main_logger.debug(dfstepclean_exploded.head())
    main_logger.debug(dfstepclean_exploded.info())
except Exception as e:
    main_logger.error("Failed to convert 'id_recipe' column to Integer.", exc_info=True)

#----------------------------------------------------------------
# 8.4 dfreference_equip dataframe
#----------------------------------------------------------------

# creating the dfreference_equip dataframe
try:
    dfreference_equip = dfstepclean_exploded[['id_recipe', 'id_step', 'id_equipment']]
    main_logger.info("Constructed reference_equip dataframe.")
    main_logger.debug(dfreference_equip.head())
    main_logger.debug(dfreference_equip.info())
except Exception as e:
    main_logger.error("Failed to create 'dfreference_equip' dataframe.", exc_info=True)

# Clean up dataframes
try:
    del dfstepclean_exploded
    del dfstepclean
    main_logger.info("Deleted intermediate dataframes.")
except Exception as e:
    main_logger.error("One or more intermediate dataframes were not found during deletion.", exc_info=True)


try:
    # Find duplicates
    duplicates = dfreference_equip[dfreference_equip.duplicated()]
    main_logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate refrence equipment. Removing duplicates.")
        dfreference_equip = dfreference_equip.drop_duplicates()
        dfreference_equip.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfreference_equip)} refrence equipment after removing duplicates.")
    else:
        main_logger.info("No duplicates found")
except Exception as e:
    main_logger.error("An error occurred while finding or removing duplicates.", exc_info=True)


2024-07-20 18:36:50,894 - __main__ - INFO - 'equipment' column in step dataframe exploded.
2024-07-20 18:36:50,896 - __main__ - INFO - Renamed column 'equipment' to 'equip_name'.
2024-07-20 18:36:50,901 - __main__ - INFO - 'id_equipment' column added according to the 'EQUIP_MAPPING'.
2024-07-20 18:36:50,903 - __main__ - INFO - 'id_equipment' column converted to Integer.
2024-07-20 18:36:50,915 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-20 18:36:50,917 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-20 18:36:50,929 - __main__ - INFO - Constructed reference_equip dataframe.
2024-07-20 18:36:50,936 - __main__ - INFO - Deleted intermediate dataframes.
2024-07-20 18:36:50,942 - __main__ - INFO - No duplicates found


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   equip_name    172 non-null    object
 1   step          334 non-null    object
 2   length        93 non-null     object
 3   number        334 non-null    int64 
 4   title         334 non-null    object
 5   id_step       334 non-null    int64 
 6   id_equipment  172 non-null    Int64 
dtypes: Int64(1), int64(2), object(4)
memory usage: 18.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   equip_name    172 non-null    object
 1   step          334 non-null    object
 2   length        93 non-null     object
 3   number        334 non-null    int64 
 4   title         334 non-null    object
 5   id_step       334 non-null    int64 
 6   id_equipment

In [52]:
dfreference_equip.head(30)

,id_recipe,id_step,id_equipment
0,51,276,23
1,51,277,3
2,51,278,<NA>
3,51,279,3
4,51,280,<NA>
5,51,281,<NA>
6,51,282,3
7,51,283,19
8,51,283,3
9,51,284,<NA>


## Dealing with dish types

In [53]:
# 9.1 Constructing the dfdish dataframe
#----------------------------------------------------------------
# Define the required columns for the dish DataFrame
dfdish_COLUMNS = ['dishTypes', 'title']
# Construct the DataFrame
try:
    dfdish = pd.DataFrame(df, columns=dfdish_COLUMNS)
    main_logger.info(f"Extracted dish types for {len(dfdish)} recipes.")
    main_logger.debug(dfdish.head())
    main_logger.debug(dfdish.info())
except Exception as e:
    main_logger.error("Failed to construct dfdish DataFrame.", exc_info=True)

# Explode the 'dishTypes' column
try:
    dfALLdish_types = dfdish.explode('dishTypes').reset_index(drop=True)
    main_logger.info(f"'dishTypes' column exploded and there are {len(dfALLdish_types)} dish types.")
    main_logger.debug(dfALLdish_types.head())
    main_logger.debug(dfALLdish_types.info())
except Exception as e:
    main_logger.error("Failed to explode 'dishTypes' column.", exc_info=True)

# Copy the exploded DataFrame
dfdish_type=dfALLdish_types.copy()

# Delete 'title' column
try:
    dfdish_type.drop(columns=['title'], inplace=True)
    main_logger.info("Deleted 'title' column.")
except Exception as e:
    main_logger.error("Failed to delete 'title' column.", exc_info=True)

try:
    # Find duplicates
    duplicates = dfdish_type[dfdish_type.duplicated()]
    main_logger.debug(duplicates)
    
    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate dish types. Removing duplicates.")
        dfdish_type = dfdish_type.drop_duplicates()
        dfdish_type.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfdish_type)} dish types after removing duplicates.")
    else:
        main_logger.info("No duplicates found.")
except Exception as e:
    main_logger.error("An error occurred while finding or removing duplicates.", exc_info=True)


try:
    # Find Null values
    none_rows = dfdish_type[dfdish_type['dishTypes'].isna()]
    main_logger.debug(none_rows)

    # Remove null values
    if not none_rows.empty:
        main_logger.warning(f"Found {len(none_rows)} null dish types. Removing null values.")
        dfdish_type.dropna(inplace=True)
        dfdish_type.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfdish_type)} dish types after removing null values.")
    else:
        main_logger.info("No null values found.")
except Exception as e:
    main_logger.error("An error occurred while finding or removing null values.", exc_info=True)

# Rename 'dishTypes' column to 'dish_type'
try:
    dfdish_type = dfdish_type.rename(columns={'dishTypes': 'dish_type'})
    main_logger.info("Renamed column 'dishTypes' to 'dish_type'.")
except Exception as e:
    main_logger.error("Failed to rename 'dishTypes' column.", exc_info=True)

#----------------------------------------------------------------
# 9.2 Reading Existing dish types Mapping
#----------------------------------------------------------------
# Function to read existing DISH_MAPPING.txt
def read_dish_mapping(file_path):
    try:
        with open(file_path, 'r') as file:
            DISH_MAPPING = json.load(file)
        return set(DISH_MAPPING.keys()), DISH_MAPPING
    except FileNotFoundError:
        main_logger.warning(f"{file_path} not found. Starting with an empty set.")
        return set(), {}
    except Exception as e:
        main_logger.error(f"Failed to read DISH_MAPPING file: {e}", exc_info=True)
        return set(), {}

# Read existing DISH_MAPPING.txt
existing_dishes, existing_dish_mapping = read_dish_mapping('DISH_MAPPING.txt')
main_logger.info("mapping read")

#----------------------------------------------------------------
# 9.3 Construct the id_dish_type column
#----------------------------------------------------------------

# Assign existing dish type IDs and handle new dish types
try:
    # Map existing dish type IDs based on the dish_type column
    dfdish_type['id_dish_type'] = dfdish_type['dish_type'].map(existing_dish_mapping)

    # Identify new dish types
    new_dish_types = dfdish_type[dfdish_type['id_dish_type'].isna()]

    # Extract a counter from which we start the id range
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 5:
            counter_dish = int(lines[5].strip())
            main_logger.info(f"Counter dish types: {counter_dish}")
        else:
            counter_dish = 1
            main_logger.info(f"Counter dish types initialized: {counter_dish}")

    # Assign new IDs to new dish types
    new_dish_types['id_dish_type'] = range(counter_dish, counter_dish + len(new_dish_types))
    main_logger.info("New IDs assigned to new dish types")

    # Combine existing and new dish types
    dfdish_type.update(new_dish_types)
    main_logger.info("New dish types IDs combined with existing dish types.")

    # Convert id_dish_type to integer type 
    dfdish_type['id_dish_type'] = dfdish_type['id_dish_type'].astype('Int64')
    main_logger.info("'id_dish_type' column converted to Integer")

except Exception as e:
    main_logger.error(f"Failed to assign IDs or combine dish types: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_dish += len(dfdish_type)
    main_logger.info(f"Counter dish types incremented: {counter_dish}")
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_dish) + "\n")
    main_logger.info(f"'counter_dish' appended to {file.name} successfully.")
except Exception as e:
    main_logger.error(f"Error appending to file: {e}", exc_info=True)


# Create a mapping dictionary from dfdish_type
try:
    new_dish_mapping = dfdish_type.set_index('dish_type')['id_dish_type'].to_dict()
    main_logger.info("Created 'new_dish_mapping'.")
except Exception as e:
    main_logger.error(f"Failed to create 'new_dish_mapping': {e}", exc_info=True)

# Update DISH_MAPPING with new dish types
try:
    DISH_MAPPING = {**existing_dish_mapping, **new_dish_mapping}
    with open('DISH_MAPPING.txt', 'w') as file:
        json.dump(DISH_MAPPING, file)
    main_logger.info("Updated and saved 'DISH_MAPPING'.")
except Exception as e:
    main_logger.error(f"Failed to update and save 'DISH_MAPPING': {e}", exc_info=True)



2024-07-20 18:36:57,298 - __main__ - INFO - Extracted dish types for 50 recipes.
2024-07-20 18:36:57,307 - __main__ - INFO - 'dishTypes' column exploded and there are 221 dish types.
2024-07-20 18:36:57,312 - __main__ - INFO - Deleted 'title' column.
2024-07-20 18:36:57,315 - __main__ - WARNING - Found 201 duplicate dish types. Removing duplicates.
2024-07-20 18:36:57,317 - __main__ - INFO - DataFrame now contains 20 dish types after removing duplicates.
2024-07-20 18:36:57,320 - __main__ - WARNING - Found 1 null dish types. Removing null values.
2024-07-20 18:36:57,322 - __main__ - INFO - DataFrame now contains 19 dish types after removing null values.
2024-07-20 18:36:57,324 - __main__ - INFO - Renamed column 'dishTypes' to 'dish_type'.
2024-07-20 18:36:57,326 - __main__ - INFO - mapping read
2024-07-20 18:36:57,329 - __main__ - INFO - Counter dish types: 24
/tmp/ipykernel_9616/965919431.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dishTypes  50 non-null     object
 1   title      50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dishTypes  220 non-null    object
 1   title      221 non-null    object
dtypes: object(2)
memory usage: 3.6+ KB


In [54]:
dfdish_type.head()

,dish_type,id_dish_type
0,lunch,1
1,soup,2
2,main course,3
3,main dish,4
4,dinner,5


## dfis_a dataframe

In [55]:
# 10.1 'id_dish_type' column
#----------------------------------------------------------------
# Define a function to map dish types to id_dish_type, handling missing values
def map_with_none(dish_type, mapping_dict):
    """Maps dish type names to id_dish_type, handling missing values."""
    if pd.isna(dish_type):
        return None  # Return None for missing dish types
    return mapping_dict.get(dish_type, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to 'dishTypes' column
try:
    dfALLdish_types['id_dish_type'] = dfALLdish_types['dishTypes'].apply(map_with_none, args=(DISH_MAPPING,))
    main_logger.info("'id_dish_type' column added according to the 'DISH_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to map 'dishTypes' to 'id_dish_type'.", exc_info=True)

# Convert 'id_dish_type' to integer type where applicable, keeping None values
try:
    dfALLdish_types['id_dish_type'] = dfALLdish_types['id_dish_type'].astype('Int64')
    main_logger.info("'id_dish_type' column converted to Integer.")
except Exception as e:
    main_logger.error("Failed to convert 'id_dish_type' to Integer.", exc_info=True)

#----------------------------------------------------------------
# 10.2 'id_recipe' column
#----------------------------------------------------------------

# Apply the custom map function to 'title' column for 'id_recipe'
try:
    dfALLdish_types['id_recipe'] = dfALLdish_types['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    main_logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to map 'title' to 'id_recipe'.", exc_info=True)

# Convert 'id_recipe' to integer type where applicable, keeping None values
try:
    dfALLdish_types['id_recipe'] = dfALLdish_types['id_recipe'].astype('Int64')
    main_logger.info("'id_recipe' column converted to Integer.")
except Exception as e:
    main_logger.error("Failed to convert 'id_recipe' to Integer.", exc_info=True)

#----------------------------------------------------------------
# 10.3 Some cleaning
#----------------------------------------------------------------

# Log the DataFrame information
main_logger.debug(dfALLdish_types.head())
main_logger.debug(dfALLdish_types.info())

# Delete 'dishTypes' and 'title' columns
try:
    dfALLdish_types.drop(columns=['dishTypes', 'title'], inplace=True)
    main_logger.info("Deleted columns 'dishTypes' and 'title'.")
except Exception as e:
    main_logger.error("Failed to delete columns 'dishTypes' or 'title'.", exc_info=True)

# Copy the DataFrame and delete the original
try:
    dfis_a = dfALLdish_types.copy()
    del dfALLdish_types
    main_logger.info("Copied dfALLdish_types to dfis_a and deleted dfALLdish_types.")
except Exception as e:
    main_logger.error("Failed to copy or delete dfALLdish_types.", exc_info=True)

# Log the final DataFrame information
main_logger.debug(dfis_a.head())
main_logger.debug(dfis_a.info())


2024-07-20 18:37:05,573 - __main__ - INFO - 'id_dish_type' column added according to the 'DISH_MAPPING'.
2024-07-20 18:37:05,578 - __main__ - INFO - 'id_dish_type' column converted to Integer.
2024-07-20 18:37:05,583 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-20 18:37:05,588 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-20 18:37:05,611 - __main__ - INFO - Deleted columns 'dishTypes' and 'title'.
2024-07-20 18:37:05,613 - __main__ - INFO - Copied dfALLdish_types to dfis_a and deleted dfALLdish_types.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dishTypes     220 non-null    object
 1   title         221 non-null    object
 2   id_dish_type  220 non-null    Int64 
 3   id_recipe     221 non-null    Int64 
dtypes: Int64(2), object(2)
memory usage: 7.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id_dish_type  220 non-null    Int64
 1   id_recipe     221 non-null    Int64
dtypes: Int64(2)
memory usage: 4.0 KB


In [56]:
dfis_a.head(30)

,id_dish_type,id_recipe
0,1,51
1,2,51
2,3,51
3,4,51
4,5,51
5,1,52
6,3,52
7,4,52
8,5,52
9,6,53


## Dealing with cuisines

In [57]:
# 11.1 Constructing the dfcuisine dataframe
#----------------------------------------------------------------
# Define the required columns for the cuisines DataFrame
dfcuisines_COLUMNS = ['cuisines', 'title']

# Construct the DataFrame
try:
    dfALLcuisines = pd.DataFrame(df, columns=dfcuisines_COLUMNS)
    main_logger.info(f"Extracted cuisines for {len(dfALLcuisines)} recipes.")
    main_logger.debug(dfALLcuisines.head())
    main_logger.debug(dfALLcuisines.info())
except Exception as e:
    main_logger.error("Failed to construct dfALLcuisines DataFrame.", exc_info=True)

# Explode the 'cuisines' column
try:
    dfALLcuisines = dfALLcuisines.explode('cuisines').reset_index(drop=True)
    main_logger.info(f"'cuisines' column exploded and contains {len(dfALLcuisines)} cuisines.")
    main_logger.debug(dfALLcuisines.info())
except Exception as e:
    main_logger.error("Failed to explode 'cuisines' column.", exc_info=True)

# Copy the exploded DataFrame
dfcuisine=dfALLcuisines.copy()

# Delete 'title' column
try:
    dfcuisine.drop(columns=['title'], inplace=True)
    main_logger.info("Deleted 'title' column.")
except Exception as e:
    main_logger.error("Failed to delete 'title' column.", exc_info=True)

try:
    # Find duplicates
    duplicates = dfcuisine[dfcuisine.duplicated()]
    main_logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        main_logger.warning(f"Found {len(duplicates)} duplicate cuisines. Removing duplicates.")
        dfcuisine = dfcuisine.drop_duplicates()
        dfcuisine.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfcuisine)} cuisines after removing duplicates.")
    else:
        main_logger.info("No duplicates found.")
except Exception as e:
    main_logger.error("An error occurred while finding or removing duplicates.", exc_info=True)

try:
    # Find Null values
    none_rows = dfcuisine[dfcuisine['cuisines'].isna()]
    main_logger.debug(none_rows)

    # Remove null values
    if not none_rows.empty:
        main_logger.warning(f"Found {len(none_rows)} null cuisine values. Removing null values.")
        dfcuisine.dropna(inplace=True)
        dfcuisine.reset_index(drop=True, inplace=True)
        main_logger.info(f"DataFrame now contains {len(dfcuisine)} cuisines after removing null values.")
    else:
        main_logger.info("No null values found.")
except Exception as e:
    main_logger.error("An error occurred while finding or removing null values.", exc_info=True)

# Rename 'cuisines' column to 'recipe_cuisine'
try:
    dfcuisine = dfcuisine.rename(columns={'cuisines': 'recipe_cuisine'})
    main_logger.info("Renamed column 'cuisines' to 'recipe_cuisine'.")
    main_logger.debug(dfcuisine.info())
except Exception as e:
    main_logger.error("Failed to rename 'cuisines' column.", exc_info=True)

#----------------------------------------------------------------
# 11.2 Reading Existing cuisines Mapping
#----------------------------------------------------------------
# Function to read existing CUISINES_MAPPING.txt
def read_cuisines_mapping(file_path):
    try:
        with open(file_path, 'r') as file:
            CUISINES_MAPPING = json.load(file)
        return set(CUISINES_MAPPING.keys()), CUISINES_MAPPING
    except FileNotFoundError:
        main_logger.warning(f"{file_path} not found. Starting with an empty set.")
        return set(), {}
    except Exception as e:
        main_logger.error(f"Failed to read CUISINES_MAPPING file: {e}", exc_info=True)
        return set(), {}

# Read existing CUISINES_MAPPING.txt
existing_cuisines, existing_cuisines_mapping = read_cuisines_mapping('CUISINES_MAPPING.txt')
main_logger.info("mapping read")

#----------------------------------------------------------------
# 11.3 Construct the 'id_cuisine' column
#----------------------------------------------------------------

# Assign existing cuisine IDs and handle new cuisines
try:
    # Map existing cuisine IDs based on the 'id_cuisine' column
    dfcuisine['id_cuisine'] = dfcuisine['recipe_cuisine'].map(existing_cuisines_mapping)

    # Identify new cuisines
    new_cuisines = dfcuisine[dfcuisine['id_cuisine'].isna()]

    # Extract a counter from which we start the id range
    with open('counters.txt', 'r') as file:
        lines = file.readlines()
        if len(lines) > 6:
            counter_cuisines = int(lines[6].strip())
            main_logger.info(f"Counter cuisines: {counter_cuisines}")
        else:
            counter_cuisines = 1
            main_logger.info(f"Counter cuisines initialized: {counter_cuisines}")

    # Assign new IDs to new cuisines
    new_cuisines['id_cuisine'] = range(counter_cuisines, counter_cuisines + len(new_cuisines))
    main_logger.info("New IDs assigned to new cuisines")

    # Combine existing and new cuisines
    dfcuisine.update(new_cuisines)
    main_logger.info("New cuisine IDs combined with existing cuisines.")

    # Convert id_cuisine to integer type 
    dfcuisine['id_cuisine'] = dfcuisine['id_cuisine'].astype('Int64')
    main_logger.info("'id_cuisine' column converted to Integer")

except Exception as e:
    main_logger.error(f"Failed to assign IDs or combine cuisines: {e}", exc_info=True)

# Append the counter content to the file
try:
    counter_cuisines += len(new_cuisines)
    main_logger.info(f"Counter cuisines incremented: {counter_cuisines}")
    with open('new_counters.txt', 'a') as file:
        file.write(str(counter_cuisines) + "\n")
    main_logger.info(f"'counter_cuisines' appended to {file.name} successfully.")
except Exception as e:
    main_logger.error(f"Error appending to file: {e}", exc_info=True)

# Create a mapping dictionary from dfcuisine
try:
    new_cuisines_mapping = dfcuisine.set_index('recipe_cuisine')['id_cuisine'].to_dict()
    main_logger.info("Created 'new_cuisines_mapping'.")
except Exception as e:
    main_logger.error(f"Failed to create 'new_cuisines_mapping': {e}", exc_info=True)

# Update CUISINES_MAPPING with new cuisines
try:
    CUISINES_MAPPING = {**existing_cuisines_mapping, **new_cuisines_mapping}
    with open('CUISINES_MAPPING.txt', 'w') as file:
        json.dump(CUISINES_MAPPING, file)
    main_logger.info("Updated and saved 'CUISINES_MAPPING'.")
except Exception as e:
    main_logger.error(f"Failed to update and save 'CUISINES_MAPPING': {e}", exc_info=True)


2024-07-20 18:37:11,484 - __main__ - INFO - Extracted cuisines for 50 recipes.
2024-07-20 18:37:11,495 - __main__ - INFO - 'cuisines' column exploded and contains 60 cuisines.
2024-07-20 18:37:11,501 - __main__ - INFO - Deleted 'title' column.
2024-07-20 18:37:11,504 - __main__ - WARNING - Found 45 duplicate cuisines. Removing duplicates.
2024-07-20 18:37:11,505 - __main__ - INFO - DataFrame now contains 15 cuisines after removing duplicates.
2024-07-20 18:37:11,507 - __main__ - WARNING - Found 1 null cuisine values. Removing null values.
2024-07-20 18:37:11,509 - __main__ - INFO - DataFrame now contains 14 cuisines after removing null values.
2024-07-20 18:37:11,510 - __main__ - INFO - Renamed column 'cuisines' to 'recipe_cuisine'.
2024-07-20 18:37:11,514 - __main__ - INFO - mapping read
2024-07-20 18:37:11,517 - __main__ - INFO - Counter cuisines: 13
/tmp/ipykernel_9616/1767101143.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cuisines  50 non-null     object
 1   title     50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cuisines  19 non-null     object
 1   title     60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   recipe_cuisine  14 non-null     object
dtypes: object(1)
memory usage: 240.0+ bytes


In [58]:
dfcuisine.head()

,recipe_cuisine,id_cuisine
0,Indian,10
1,Asian,4
2,English,13
3,Scottish,14
4,British,15


## dfbelongs dataframe

In [59]:
# 12.1 'id_cuisine' column
#----------------------------------------------------------------

# Define a function to map recipe cuisines to id_cuisine, handling missing values
def map_with_none(recipe_cuisine, CUISINES_MAPPING):
    """Maps recipes cuisines to id_cuisine, handling missing values."""
    if pd.isna(recipe_cuisine):
        return None  # Return None for missing cuisines
    return CUISINES_MAPPING.get(recipe_cuisine, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to add 'id_cuisine' column
try:
    dfALLcuisines['id_cuisine'] = dfALLcuisines['cuisines'].apply(map_with_none, args=(CUISINES_MAPPING,))
    main_logger.info("'id_cuisine' column added according to the 'CUISINES_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to map 'id_cuisine' to 'cuisines'.", exc_info=True)

# Convert id_cuisine to integer type where applicable, keeping None values
try:
    dfALLcuisines['id_cuisine'] = dfALLcuisines['id_cuisine'].astype('Int64')
    main_logger.info("'id_cuisine' column converted to Integer ")
except Exception as e:
    main_logger.error("Failed to convert 'id_cuisine' to Integer.", exc_info=True)

#----------------------------------------------------------------
# 11.2 'id_recipe' column
#----------------------------------------------------------------

# Apply the custom map function to add 'id_recipe' column
try:
    dfALLcuisines['id_recipe'] = dfALLcuisines['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    main_logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    main_logger.error("Failed to map 'id_recipe' to 'title'.", exc_info=True)

# Convert id_recipe to integer type where applicable, keeping None values
try:
    dfALLcuisines['id_recipe'] = dfALLcuisines['id_recipe'].astype('Int64')
    main_logger.info("'id_recipe' column converted to Integer.")
except Exception as e:
    main_logger.error("Failed to convert 'id_recipe' to Integer.", exc_info=True)

#----------------------------------------------------------------
# 10.3 Some cleaning
#----------------------------------------------------------------

# Log the DataFrame head and info
main_logger.debug(dfALLcuisines.head())
main_logger.debug(dfALLcuisines.info())

# Drop 'cuisines' and 'title' columns from dfcuisines
try:
    dfALLcuisines.drop(columns=['cuisines', 'title'], inplace=True)
    main_logger.info("Deleted 'cuisines' and 'title' columns.")
except Exception as e:
    main_logger.error("Failed to delete 'cuisines' or 'title' columns.", exc_info=True)

# Copy dfcuisines to dfbelongs and delete dfcuisines
try:
    dfbelongs = dfALLcuisines.copy()
    main_logger.info("Copied dfALLcuisines to dfbelongs.")
    
    del dfALLcuisines
    main_logger.info("Deleted dfALLcuisines.")

    main_logger.debug(dfbelongs.head())
    main_logger.debug(dfbelongs.info())
except Exception as e:
    main_logger.error("Failed to copy or delete dfcuisines.", exc_info=True)


2024-07-20 18:37:17,123 - __main__ - INFO - 'id_cuisine' column added according to the 'CUISINES_MAPPING'.
2024-07-20 18:37:17,125 - __main__ - INFO - 'id_cuisine' column converted to Integer 
2024-07-20 18:37:17,127 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-20 18:37:17,129 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-20 18:37:17,146 - __main__ - INFO - Deleted 'cuisines' and 'title' columns.
2024-07-20 18:37:17,147 - __main__ - INFO - Copied dfALLcuisines to dfbelongs.
2024-07-20 18:37:17,150 - __main__ - INFO - Deleted dfALLcuisines.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cuisines    19 non-null     object
 1   title       60 non-null     object
 2   id_cuisine  19 non-null     Int64 
 3   id_recipe   60 non-null     Int64 
dtypes: Int64(2), object(2)
memory usage: 2.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_cuisine  19 non-null     Int64
 1   id_recipe   60 non-null     Int64
dtypes: Int64(2)
memory usage: 1.2 KB


In [60]:
dfbelongs.head(50)

,id_cuisine,id_recipe
0,<NA>,51
1,<NA>,52
2,<NA>,53
3,<NA>,54
4,<NA>,55
5,<NA>,56
6,<NA>,57
7,<NA>,58
8,<NA>,59
9,<NA>,60


## Transform Recap

In [ ]:
main_logger.debug("Displaying transforamtion recap")

### recipe table
(id_recipe INT, recipe_title VARCHAR(50), ready_min INT, summary VARCHAR(2000), servings INT, is_cheap LOGICAL, price_per_serving DOUBLE, is_vegetarian LOGICAL, is_vegan LOGICAL, is_glutenFree LOGICAL, is_dairyFree LOGICAL, is_healthy LOGICAL, is_sustainable LOGICAL, is_lowFodmap LOGICAL, is_Popular LOGICAL, license VARCHAR(20), source_url VARCHAR(100));

In [ ]:
main_logger.debug(dfrecipes.info())

### Ingredients table
(id_ingredient INT, ing_name VARCHAR(50), consistency VARCHAR(20), aisle VARCHAR(20));

In [ ]:
main_logger.debug(dfIng.info())

### reference_ing table
(#id_recipe, #id_ingredient, measure VARCHAR(50));

In [ ]:
main_logger.debug(dfreference_ing.info())

### Equipment table
(id_equipment INT, equip_name VARCHAR(50));

In [ ]:
main_logger.debug(dfequip.info())

### Instructions table
Instruction = (id_instruction INT, #id_recipe);

In [ ]:
main_logger.debug(dfIns.info())

### steps table
(id_step INT, step VARCHAR(8000), number INT, length VARCHAR(50), #id_instruction);

In [ ]:
main_logger.debug(dfstep_final.info())

### reference_equip table
(#id_recipe, #id_step, #id_equipment);

In [ ]:
main_logger.debug(dfreference_equip.info())

### dish_type table
(id_dish_type INT, dish_type VARCHAR(50))

In [ ]:
main_logger.debug(dfdish_type.info())

### is_a  table
(#id_recipe, #id_dish_type)

In [ ]:
main_logger.debug(dfis_a.info())

### cuisine table
(id_cuisine INT, recipe_cuisine VARCHAR(50));

In [ ]:
main_logger.debug(dfcuisine.info())

### belongs table
(#id_recipe, #id_cuisine);

In [ ]:
main_logger.debug(dfbelongs.info())

# load

In [61]:
# Database configuration
DB_HOST='10.0.2.15' # Database host IP address
DB_PORT= 5432 # Database port number
DB_NAME='recipe_etl' # Name of the database
DB_USER='maryem' # Database user name
DB_PASSWORD='HelloWorld' # Database user password

try:
    # Establish connection to the database
    conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
    main_logger.info(f"Connected to {DB_NAME}")
except psycopg2.Error as e:
    main_logger.error(f"Error: Could not make connection to the database {DB_NAME}")
    main_logger.error(e)
    conn = None

2024-07-20 19:01:50,310 - __main__ - INFO - Connected to recipe_etl


In [62]:
# Function to check if a record already exists in the database
def record_exists(query, values):
    """
    Check if a record already exists in the database.

    Parameters:
    query (str): The SQL query to execute.
    values (tuple): The values to use in the SQL query.

    Returns:
    bool: True if the record exists, False otherwise.
    """
    try:
        # Execute the query with the provided values
        cur.execute(query, values)
        # Check if any record was fetched (exists)
        return cur.fetchone() is not None
    except psycopg2.Error as e:
        # Log an error if any occurs during the check
        main_logger.error(f"Error in checking for existing record: {e}")
        return False

# Function to execute an insert query and check if the insertion was successful
def execute_insert_and_check(query, values, table_name, check_query, check_values):
    """
    Execute an insert query and check if the insertion was successful.

    Parameters:
    query (str): The SQL insert query to execute.
    values (tuple): The values to use in the SQL insert query.
    table_name (str): The name of the table to insert into.
    check_query (str): The SQL query to check if the record already exists.
    check_values (tuple): The values to use in the check query.

    Raises:
    ValueError: If the table is empty after insertion attempts.
    """
    try:
        # Check if the record already exists
        if not record_exists(check_query, check_values):
            # Execute the insert query with the provided values
            cur.execute(query, values)
            # Commit the transaction
            conn.commit()
            # Check the number of rows in the table after the insert
            cur.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cur.fetchone()[0]
            # Raise an error if the table is empty after insertion
            if row_count == 0:
                main_logger.error(f"The {table_name} table is empty after insertion attempts.")
                raise ValueError(f"The {table_name} table is empty after insertion attempts.")
            else:
                # Log success if insertion was successful
                main_logger.info(f"Values inserted into {table_name} successfully")
        else:
            # Log a warning if a duplicate entry is found
            main_logger.warning(f"Duplicate entry found in {table_name}. Skipping insertion.")
    except psycopg2.Error as e:
        # Log an error if any occurs during the insertion
        main_logger.error(f"Error in inserting values into {table_name}: {e}", exc_info=True)
        # Revert the database to the state it was in before the current transaction began.
        conn.rollback()
        raise
    except ValueError as ve:
        # Log any value errors
        main_logger.error(ve)
        raise


In [63]:
# Proceed only if the connection was successful
if conn:
    try:
        # Obtain a cursor to execute queries
        cur = conn.cursor()
        # Log that the cursor was obtained successfully
        main_logger.info(f"Cursor obtained for the database {DB_NAME}")
    except psycopg2.Error as e:
        # Log an error if the cursor could not be obtained
        main_logger.error(f"Error: Could not get cursor for the database {DB_NAME}")
        main_logger.error(e)
        cur = None
    
    # If cursor is obtained successfully
    if cur:
        # Insert into Recipe table
        try:
            id_recipe_set=set()
            # Iterate over each row in the dataframe dfrecipes
            for index, row in dfrecipes.iterrows():
                # Check if the recipe already exists by id_recipe or recipe_title
                check_query = """
                SELECT 1 FROM Recipe WHERE recipe_title = %s
                """
                check_values = (row['recipe_title'],)

                # Execute the insert and check function for the Recipe table
                execute_insert_and_check(
                    """
                    INSERT INTO Recipe (id_recipe, recipe_title, ready_min, summary, servings, is_cheap, price_per_serving, 
                    is_vegetarian, is_vegan, is_glutenFree, is_dairyFree, is_healthy, is_sustainable, is_lowFodmap, 
                    is_Popular, license, source_url)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",
                    (
                        int(row['id_recipe']),
                        row['recipe_title'],
                        int(row['ready_min']),
                        row['summary'],
                        int(row['servings']),
                        bool(row['is_cheap']),
                        float(row['price_per_serving']),
                        bool(row['is_vegetarian']),
                        bool(row['is_vegan']),
                        bool(row['is_glutenFree']),
                        bool(row['is_dairyFree']),
                        bool(row['is_healthy']),
                        bool(row['is_sustainable']),
                        bool(row['is_lowFodmap']),
                        bool(row['is_Popular']),
                        row['license'],
                        row['source_url'],
                    ),
                    'Recipe',
                    check_query,
                    check_values
                )
    
                # Check if the recipe was inserted
                cur.execute(check_query, check_values)
                if not cur.fetchone():
                    id_recipe_set.add(int(row['id_recipe'])) # Save the recipe ID for related table insertions
                    continue  # Skip to the next recipe if this one already exists
           
            # Insert into Instruction table
            try:
                id_ins_set=set()
                for index, row in dfIns.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO Instruction (id_instruction, id_recipe)
                            VALUES (%s, %s);""",
                            (
                                int(row['instruction_id']),
                                int(row['id_recipe'])
                            ),
                            'Instruction',
                            "SELECT 1 FROM Instruction WHERE id_instruction = %s AND id_recipe = %s",
                            (int(row['instruction_id']), int(row['id_recipe']))
                        )
                    # Check if the instruction was inserted
                    cur.execute(check_query, check_values)
                    if not cur.fetchone():
                        id_ins_set.add(int(row['instruction_id'])) # Save the instruction ID for related table insertions
                        continue  # Skip to the next instruction if this one already exists
            except Exception as e:
                main_logger.error(f"Failed to insert into Instruction table: {e}", exc_info=True)
            
            # Insert into Ingredient table
            try:
                id_ing_set = set()
                for index, row in dfIng.iterrows():
                    # Check if the ingredient already exists by ing_name
                    check_query = """
                    SELECT 1 FROM Ingredient WHERE ing_name = %s
                    """
                    check_values = (row['ing_name'],)

                    execute_insert_and_check(
                        """
                        INSERT INTO Ingredient (id_ingredient, ing_name, consistency, aisle)
                        VALUES (%s, %s, %s, %s);""",
                        (
                            int(row['id_ingredient']),
                            row['ing_name'],
                            row['consistency'],
                            row['aisle']
                        ),
                        'Ingredient',
                        check_query,
                        check_values
                    )

                    # Check if the ingredient was inserted
                    cur.execute(check_query, check_values)
                    if cur.fetchone():
                        continue  # Skip to the next ingredient if this one already exists
                    cur.execute("""SELECT id_ingredient FROM Ingredient WHERE ing_name = %s""", (row['ing_name'],))
                    # Fetch the result
                    result = cur.fetchone()
                    # Extract the id_ingredient from the result
                    old_id = result[0]
                    # Append to the set
                    id_ing_set.add({int(row['id_ingredient']): old_id})

                # Initialize an empty dictionary to hold the final combined dictionary
                ing_id_MAP = {}
                # Iterate over the set and update the final dictionary with each dictionary
                for d in id_ing_set:
                    ing_id_MAP.update(d)
            except Exception as e:
                main_logger.error(f"Failed to insert into Ingredient table: {e}", exc_info=True)

            # Insert into Step table
            try:
                id_step_set=set()
                for index, row in dfstep_final.iterrows():
                    if int(row['instruction_id']) not in id_ins_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO Step (id_step, step, number, length, id_instruction)
                            VALUES (%s, %s, %s, %s, %s);""",
                            (
                                int(row['id_step']),
                                row['step'],
                                int(row['number']),
                                row['length'],
                                int(row['instruction_id'])
                            ),
                            'Step',
                            "SELECT 1 FROM Step WHERE id_step = %s",
                            (int(row['id_step']),)
                    )
                    # Check if the step was inserted
                    cur.execute(check_query, check_values)
                    if not cur.fetchone():
                        id_recipe_set.add(int(row['id_step'])) # Save the step ID for related table insertions
                        continue  # Skip to the next step if this one already exists
            except Exception as e:
                main_logger.error(f"Failed to insert into Step table: {e}", exc_info=True)

            # Insert into Equipment table
            try:
                for index, row in dfequip.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Equipment (id_equipment, equip_name)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_equipment']),
                            row['equip_name']
                        ),
                        'Equipment',
                        "SELECT 1 FROM Equipment WHERE id_equipment = %s",
                        (int(row['id_equipment']),)
                    )
            except Exception as e:
                main_logger.error(f"Failed to insert into Equipment table: {e}", exc_info=True)

            # Insert into Dish table
            try:
                for index, row in dfdish_type.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Dish (id_dish_type, dish_type)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_dish_type']),
                            row['dish_type']
                        ),
                        'Dish',
                        "SELECT 1 FROM Dish WHERE id_dish_type = %s",
                        (int(row['id_dish_type']),)
                    )
            except Exception as e:
                main_logger.error(f"Failed to insert into Dish table: {e}", exc_info=True)

            # Insert into Cuisine table
            try:
                for index, row in dfcuisine.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Cuisine (id_cuisine, recipe_cuisine)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_cuisine']),
                            row['recipe_cuisine']
                        ),
                        'Cuisine',
                        "SELECT 1 FROM Cuisine WHERE id_cuisine = %s",
                        (int(row['id_cuisine']),)
                    )
            except Exception as e:
                main_logger.error(f"Failed to insert into Cuisine table: {e}", exc_info=True)

            # Insert into reference_ing table
            try:
                dfreference_ing = dfreference_ing.replace({pd.NA: np.nan})
                for index, row in dfreference_ing.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        ing_id= int(row['id_ingredient'])
                        if ing_id in ing_id_MAP.keys():
                            ing_id=ing_id_MAP.get(ing_id)
                            main_logger.warning("altered ingredient id because it exists")
                        execute_insert_and_check(
                            """
                            INSERT INTO reference_ing (id_recipe, id_ingredient, measure)
                            VALUES (%s, %s, %s);""",
                            (
                                int(row['id_recipe']),
                                ing_id,
                                row['measure']
                            ),
                            'reference_ing',
                            "SELECT 1 FROM reference_ing WHERE id_recipe = %s AND id_ingredient = %s",
                            (int(row['id_recipe']), ing_id)
                        )
            except Exception as e:
                main_logger.error(f"Failed to insert into reference_ing table: {e}", exc_info=True)

            # Insert into reference_equip table
            try:
                dfreference_equip = dfreference_equip.replace({pd.NA: np.nan})
                for index, row in dfreference_equip.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO reference_equip (id_recipe, id_step, id_equipment)
                            VALUES (%s, %s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_step']) if pd.notna(row['id_step']) else None,
                                int(row['id_equipment']) if pd.notna(row['id_equipment']) else None
                            ),
                            'reference_equip',
                            "SELECT 1 FROM reference_equip WHERE id_recipe = %s AND id_step = %s AND id_equipment = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_step']) if pd.notna(row['id_step']) else None,
                                int(row['id_equipment']) if pd.notna(row['id_equipment']) else None
                            )
                            )
            except Exception as e:
                main_logger.error(f"Failed to insert into reference_equip table: {e}", exc_info=True)

            # Insert into belongs table
            try:
                dfbelongs = dfbelongs.replace({pd.NA: np.nan})
                for index, row in dfbelongs.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO belongs (id_recipe, id_cuisine)
                            VALUES (%s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_cuisine']) if pd.notna(row['id_cuisine']) else None
                            ),
                            'belongs',
                            "SELECT 1 FROM belongs WHERE id_recipe = %s AND id_cuisine = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_cuisine']) if pd.notna(row['id_cuisine']) else None
                            )
                            )
            except Exception as e:
                logger.error(f"Failed to insert into belongs table: {e}", exc_info=True)

            # Insert into is_a table
            try:
                # Replace pd.NA with np.nan
                dfis_a = dfis_a.replace({pd.NA: np.nan})
                for index, row in dfis_a.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO is_a (id_recipe, id_dish_type)
                            VALUES (%s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_dish_type']) if pd.notna(row['id_dish_type']) else None
                            ),
                            'is_a',
                            "SELECT 1 FROM is_a WHERE id_recipe = %s AND id_dish_type = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_dish_type']) if pd.notna(row['id_dish_type']) else None
                            )
                            )
            except Exception as e:
                main_logger.error(f"Failed to insert into is_a table: {e}", exc_info=True)

        finally:
            cur.close()
    else:
        main_logger.error("Cursor could not be obtained. Exiting the program.")
    conn.close()
else:
    main_logger.error("Connection to the database could not be established. Exiting the program.")


2024-07-20 19:01:56,741 - __main__ - INFO - Cursor obtained for the database recipe_etl
2024-07-20 19:01:56,751 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,767 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,772 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,777 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,782 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,787 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,802 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,809 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,814 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,819 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-20 19:01:56,823 - __main__ - INFO - Values inserted in